In [1]:
import argparse
import os
import torch

from src.datasets import get_dataloader
from src.lightning import DDPM
from src.molecule_builder import get_bond_order
from src.visualizer import save_xyz_file, visualize_chain
from tqdm.auto import tqdm
from pdb import set_trace
import sys #@mastro
from src import const #@mastro
import numpy as np #@mastro
from numpy.random import default_rng
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import directed_hausdorff

from sklearn.decomposition import PCA
from src.visualizer import load_molecule_xyz, load_xyz_files
import matplotlib.pyplot as plt
import imageio

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Simulate command-line arguments
sys.argv = [
    'ipykernel_launcher.py',
    '--checkpoint', 'models/zinc_difflinker.ckpt',
    '--chains', 'trajectories',
    '--data', 'datasets',
    '--prefix', 'zinc_final_test',
    '--keep_frames', '10',
    '--device', 'cuda:0'
]

parser = argparse.ArgumentParser()
parser.add_argument('--checkpoint', action='store', type=str, required=True)
parser.add_argument('--chains', action='store', type=str, required=True)
parser.add_argument('--prefix', action='store', type=str, required=True)
parser.add_argument('--data', action='store', type=str, required=False, default=None)
parser.add_argument('--keep_frames', action='store', type=int, required=True)
parser.add_argument('--device', action='store', type=str, required=True)
args = parser.parse_args()

experiment_name = args.checkpoint.split('/')[-1].replace('.ckpt', '')
chains_output_dir = os.path.join(args.chains, experiment_name, args.prefix, 'chains')
final_states_output_dir = os.path.join(args.chains, experiment_name, args.prefix, 'final_states')
os.makedirs(chains_output_dir, exist_ok=True)
os.makedirs(final_states_output_dir, exist_ok=True)

# Loading model form checkpoint (all hparams will be automatically set)
model = DDPM.load_from_checkpoint(args.checkpoint, map_location=args.device)

# Possibility to evaluate on different datasets (e.g., on CASF instead of ZINC)
model.val_data_prefix = args.prefix

# In case <Anonymous> will run my model or vice versa
if args.data is not None:
    model.data_path = args.data

model = model.eval().to(args.device)
model.setup(stage='val')
dataloader = get_dataloader(
    model.val_dataset,
    batch_size=1, #@mastro, it was 32
    # batch_size=len(model.val_dataset)
)


c:\Users\Mastro\anaconda3\envs\diff_explainer\lib\site-packages\lightning_fabric\utilities\cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Lightning automatically upgraded your

In [2]:
def compute_molecular_similarity(mol1, mol2, mask1 = None, mask2 = None):
    """
    Compute the similarity between two molecules based on distances and atom type.
    
    Args:
        mol1 (torch.Tensor): The first molecule.
        mol2 (torch.Tensor): The second molecule.
        mask (torch.Tensor, optional): A mask indicating which atoms to consider. If not provided, all atoms will be considered.
        
    Returns:
        float: The similarity between the two molecules.
    """
    # If fragmen_mask is provided, only consider the atoms in the mask
    if mask1 is not None:
        mask1 = mask1.bool()
        mol1 = mol1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        mol2 = mol2[mask2,:]

    return 1 - torch.norm(mol1 - mol2)

def compute_molecular_distance(mol1, mol2, mask1 = None, mask2 = None):
    """
    Compute the similarity between two molecules based on distances and atom type.
    
    Args:
        mol1 (torch.Tensor): The first molecule.
        mol2 (torch.Tensor): The second molecule.
        mask (torch.Tensor, optional): A mask indicating which atoms to consider. If not provided, all atoms will be considered.
        
    Returns:
        float: The similarity between the two molecules.
    """
    # If fragmen_mask is provided, only consider the atoms in the mask
    if mask1 is not None:
        mask1 = mask1.bool()
        mol1 = mol1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        mol2 = mol2[mask2,:]

    return torch.norm(mol1 - mol2).item()

def compute_cosine_similarity(mol1, mol2, mask1 = None, mask2 = None):
    """
    Compute the similarity between two molecules based on distances and atom type.
    
    Args:
        mol1 (torch.Tensor): The first molecule.
        mol2 (torch.Tensor): The second molecule.
        mask (torch.Tensor, optional): A mask indicating which atoms to consider. If not provided, all atoms will be considered.
        
    Returns:
        float: The similarity between the two molecules.
    """
    # If fragmen_mask is provided, only consider the atoms in the mask
    if mask1 is not None:
        mask1 = mask1.bool()
        mol1 = mol1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        mol2 = mol2[mask2,:]

    return cosine_similarity(mol1.flatten().reshape(1, -1), mol2.flatten().reshape(1, -1)).item()

def compute_molecular_similarity_positions(mol1, mol2, mask1 = None, mask2 = None):
    """
    Compute the similarity between two molecules based on positions.
    
    Args:
        mol1 (torch.Tensor): The first molecule.
        mol2 (torch.Tensor): The second molecule.
        mask (torch.Tensor, optional): A mask indicating which atoms to consider. If not provided, all atoms will be considered.
        
    Returns:
        float: The similarity between the two molecules.
    """
    # If fragmen_mask is provided, only consider the atoms in the mask
    positions1 = mol1[:, :3].squeeze()
    positions2 = mol2[:, :3].squeeze()

    if mask1 is not None:
        mask1 = mask1.bool()
        positions1 = positions1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        positions2 = positions2[mask2,:]


    return 1 - torch.norm(positions1 - positions2) #choose if distance or similarity, need to check what it the better choice

def compute_one_hot_similarity(mol1, mol2, mask1 = None, mask2 = None):
    """
    Computes the similarity between two one-hot encoded molecules. The one-hot encoding indicates the atom type
    
    Args:
        mol1 (torch.Tensor): The first one-hot encoded molecule.
        mol2 (torch.Tensor): The second one-hot encoded molecule.
        mask (torch.Tensor, optional): A mask to apply on the atoms. Defaults to None.
    
    Returns:
        torch.Tensor: The similarity between the two molecules.
    """
    
    # Apply mask if provided
    if mask1 is not None:
        mask1 = mask1.bool()
        mol1 = mol1[mask1,:]

    if mask2 is not None:
        mask2 = mask2.bool()
        mol2 = mol2[mask2,:]
    
    # Compute similarity by comparing the one-hot encoded features
    similarity = torch.sum(mol1[:,3:-1] == mol2[:,3:-1]) / mol1[:, 3:-1].numel()
    
    return similarity

## Explainability

### Utility function for visualization purposes

In [3]:
def draw_sphere_xai(ax, x, y, z, size, color, alpha):
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)

    xs = size * np.outer(np.cos(u), np.sin(v))
    ys = size * np.outer(np.sin(u), np.sin(v)) #* 0.8
    zs = size * np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x + xs, y + ys, z + zs, rstride=2, cstride=2, color=color, alpha=alpha)

def plot_molecule_xai(ax, positions, atom_type, alpha, spheres_3d, hex_bg_color, is_geom, fragment_mask=None, phi_values=None):
    x = positions[:, 0]
    y = positions[:, 1]
    z = positions[:, 2]
    # Hydrogen, Carbon, Nitrogen, Oxygen, Flourine

    idx2atom = const.GEOM_IDX2ATOM if is_geom else const.IDX2ATOM

    colors_dic = np.array(const.COLORS)
    radius_dic = np.array(const.RADII)
    area_dic = 1500 * radius_dic ** 2

    areas = area_dic[atom_type]
    radii = radius_dic[atom_type]
    colors = colors_dic[atom_type]

    if fragment_mask is None:
        fragment_mask = torch.ones(len(x))

    for i in range(len(x)):
        for j in range(i + 1, len(x)):
            p1 = np.array([x[i], y[i], z[i]])
            p2 = np.array([x[j], y[j], z[j]])
            dist = np.sqrt(np.sum((p1 - p2) ** 2))
            atom1, atom2 = idx2atom[atom_type[i]], idx2atom[atom_type[j]]
            draw_edge_int = get_bond_order(atom1, atom2, dist)
            line_width = (3 - 2) * 2 * 2
            draw_edge = draw_edge_int > 0
            if draw_edge:
                if draw_edge_int == 4:
                    linewidth_factor = 1.5
                else:
                    linewidth_factor = 1
                linewidth_factor *= 0.5
                ax.plot(
                    [x[i], x[j]], [y[i], y[j]], [z[i], z[j]],
                    linewidth=line_width * linewidth_factor * 2,
                    c=hex_bg_color,
                    alpha=alpha
                )

    # from pdb import set_trace
    # set_trace()

    if spheres_3d:
        # idx = torch.where(fragment_mask[:len(x)] == 0)[0]
        # ax.scatter(
        #     x[idx],
        #     y[idx],
        #     z[idx],
        #     alpha=0.9 * alpha,
        #     edgecolors='#FCBA03',
        #     facecolors='none',
        #     linewidths=2,
        #     s=900
        # )
        for i, j, k, s, c, f, phi in zip(x, y, z, radii, colors, fragment_mask, phi_values):
            if f == 1:
                alpha = 1.0
                if phi > 0:
                    c = 'red'

            draw_sphere_xai(ax, i.item(), j.item(), k.item(), 0.5 * s, c, alpha)

    else:
        phi_values_array = np.array(list(phi_values.values()))

        # #draw fragments
        # fragment_mask_on_cpu = fragment_mask.cpu().numpy()
        # colors_fragment = colors[fragment_mask_on_cpu == 1]
        # x_fragment = x[fragment_mask_on_cpu == 1]
        # y_fragment = y[fragment_mask_on_cpu == 1]
        # z_fragment = z[fragment_mask_on_cpu == 1]
        # areas_fragment = areas[fragment_mask_on_cpu == 1]
        # ax.scatter(x_fragment, y_fragment, z_fragment, s=areas_fragment, alpha=0.9 * alpha, c=np.where(phi_values_array > 0, 'red', colors_fragment))

        # #draw non-fragment atoms
        # colors = colors[fragment_mask_on_cpu == 0]
        # x = x[fragment_mask_on_cpu == 0]
        # y = y[fragment_mask_on_cpu == 0]
        # z = z[fragment_mask_on_cpu == 0]
        # areas = areas[fragment_mask_on_cpu == 0]
        # ax.scatter(x, y, z, s=areas, alpha=0.9 * alpha, c=colors)

        #draw fragments
        fragment_mask_on_cpu = fragment_mask.cpu().numpy()
        colors_fragment = colors[fragment_mask_on_cpu == 1]
        x_fragment = x[fragment_mask_on_cpu == 1]
        y_fragment = y[fragment_mask_on_cpu == 1]
        z_fragment = z[fragment_mask_on_cpu == 1]
        areas_fragment = areas[fragment_mask_on_cpu == 1]
        
        # Calculate the gradient colors based on phi values
        cmap = plt.cm.get_cmap('coolwarm')
        norm = plt.Normalize(vmin=min(phi_values_array), vmax=max(phi_values_array))
        colors_fragment_shadow = cmap(norm(phi_values_array))
        
        # ax.scatter(x_fragment, y_fragment, z_fragment, s=areas_fragment, alpha=0.9 * alpha, c=colors_fragment)

        ax.scatter(x_fragment, y_fragment, z_fragment, s=areas_fragment, alpha=0.9 * alpha, c=colors_fragment, edgecolors=colors_fragment_shadow, linewidths=5, rasterized=False)

        #draw non-fragment atoms
        colors = colors[fragment_mask_on_cpu == 0]
        x = x[fragment_mask_on_cpu == 0]
        y = y[fragment_mask_on_cpu == 0]
        z = z[fragment_mask_on_cpu == 0]
        areas = areas[fragment_mask_on_cpu == 0]
        ax.scatter(x, y, z, s=areas, alpha=0.9 * alpha, c=colors, rasterized=False)


def plot_data3d_xai(positions, atom_type, is_geom, camera_elev=0, camera_azim=0, save_path=None, spheres_3d=False,
                bg='black', alpha=1., fragment_mask=None, phi_values=None):
    black = (0, 0, 0)
    white = (1, 1, 1)
    hex_bg_color = '#FFFFFF' if bg == 'black' else '#000000' #'#666666'

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(projection='3d')
    ax.set_aspect('auto')
    ax.view_init(elev=camera_elev, azim=camera_azim)
    if bg == 'black':
        ax.set_facecolor(black)
    else:
        ax.set_facecolor(white)
    ax.xaxis.pane.set_alpha(0)
    ax.yaxis.pane.set_alpha(0)
    ax.zaxis.pane.set_alpha(0)
    ax._axis3don = False

    if bg == 'black':
        ax.w_xaxis.line.set_color("black")
    else:
        ax.w_xaxis.line.set_color("white")

    plot_molecule_xai(
        ax, positions, atom_type, alpha, spheres_3d, hex_bg_color, is_geom=is_geom, fragment_mask=fragment_mask, phi_values=phi_values
    )

    max_value = positions.abs().max().item()
    axis_lim = min(40, max(max_value / 1.5 + 0.3, 3.2))
    ax.set_xlim(-axis_lim, axis_lim)
    ax.set_ylim(-axis_lim, axis_lim)
    ax.set_zlim(-axis_lim, axis_lim)
    dpi = 300 if spheres_3d else 300 #it was 120 and 50

    if save_path is not None:
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0.0, dpi=dpi)
        # plt.savefig(save_path, bbox_inches='tight', pad_inches=0.0, dpi=dpi, transparent=True)

        if spheres_3d:
            img = imageio.imread(save_path)
            img_brighter = np.clip(img * 1.4, 0, 255).astype('uint8')
            imageio.imsave(save_path, img_brighter)
    else:
        plt.show()
    plt.close()

def visualize_chain_xai(
        path, spheres_3d=False, bg="black", alpha=1.0, wandb=None, mode="chain", is_geom=False, fragment_mask=None, phi_values=None
):
    files = load_xyz_files(path)
    save_paths = []

    # Fit PCA to the final molecule – to obtain the best orientation for visualization
    positions, one_hot, charges = load_molecule_xyz(files[-1], is_geom=is_geom)
    pca = PCA(n_components=3)
    pca.fit(positions)

    for i in range(len(files)):
        file = files[i]

        positions, one_hot, charges = load_molecule_xyz(file, is_geom=is_geom)
        atom_type = torch.argmax(one_hot, dim=1).numpy()

        # Transform positions of each frame according to the best orientation of the last frame
        positions = pca.transform(positions)
        positions = torch.tensor(positions)

        fn = file[:-4] + '.png'
        plot_data3d_xai(
            positions, atom_type,
            save_path=fn,
            spheres_3d=spheres_3d,
            alpha=alpha,
            bg=bg,
            camera_elev=90,
            camera_azim=90,
            is_geom=is_geom,
            fragment_mask=fragment_mask,
            phi_values=phi_values
        )
        save_paths.append(fn)

    imgs = [imageio.imread(fn) for fn in save_paths]
    dirname = os.path.dirname(save_paths[0])
    gif_path = dirname + '/output.gif'
    imageio.mimsave(gif_path, imgs, subrectangles=True)

    if wandb is not None:
        wandb.log({mode: [wandb.Video(gif_path, caption=gif_path)]})

### Explainabiliy phase

In [4]:
#@mastro
num_samples = 5
sampled = 0
#end @mastro
start = 0
bond_order_dict = {0:0, 1:0, 2:0, 3:0}
ATOM_SAMPLER = False
SAVE_VISUALIZATION = True
chain_with_full_fragments = None
M = 100 #number of Monte Carlo Sampling steps
P = 0.2 #probability of atom to exist in random graph (also edge in the future)
SEED = 42 #seed for random sampling
# Create the folder if it does not exist
folder_save_path = "results/explanations"
if not os.path.exists(folder_save_path):
    os.makedirs(folder_save_path)

for data in dataloader:
    
    if sampled < num_samples:
        chain_with_full_fragments = None
        sampled += 1
        
        # generate chain with original and full fragments

        chain_batch, node_mask = model.sample_chain(data, keep_frames=args.keep_frames)
        
        #get the generated molecule and store it in a variable
        chain_with_full_fragments = chain_batch[0, 0, :, :] #need to get only the final frame, is 0 ok in the first dimension?

        # Compute distance of two chains
        mol_similarity = compute_molecular_similarity(chain_with_full_fragments.squeeze(), chain_with_full_fragments.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data["linker_mask"][0].squeeze())
        print("Similarity between the two chains:", mol_similarity.item())
        # compute similarity of one-hot vectors
        positional_similarity = compute_molecular_similarity_positions(chain_with_full_fragments.squeeze(), chain_with_full_fragments.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data["linker_mask"][0].squeeze())
        print("Similarity between the two chains based on positions:", positional_similarity.item())
        one_hot_similarity = compute_one_hot_similarity(chain_with_full_fragments.squeeze(), chain_with_full_fragments.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data["linker_mask"][0].squeeze())
        print("Similarity between the two one-hot vectors:", one_hot_similarity.item())
        # compute cosine similarity
        cos_simil = compute_cosine_similarity(chain_with_full_fragments.squeeze().cpu(), chain_with_full_fragments.squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data["linker_mask"][0].squeeze().cpu())
        print("Cosine similarity between the two chains:", cos_simil)
    
        
        # display(data["fragment_mask"])
        # display(data["fragment_mask"].shape)

        # display(data["linker_mask"])
        # display(data["linker_mask"].shape)
        
        # display(data["edge_mask"])
        # display(data["edge_mask"].shape)

        #mask out all edges that are not bonds
        # idx2atom = const.GEOM_IDX2ATOM if model.is_geom else const.IDX2ATOM
      
        # positions = data["positions"][0].detach().cpu().numpy()
        # x  = positions[:,0]
        # y  = positions[:,1]
        # z  = positions[:,2]
        # # print(x)
       
        # atom_type = torch.argmax(data["one_hot"][0], dim=1)
        # print("Number of edges", len(x) * len(x))
        # sys.exit()
        #uncomment to work on edge_mask (not huge effect, tho)
        # for i in range(len(x)):
        #     for j in range(i+1, len(x)):
        #         p1 = np.array([x[i], y[i], z[i]])
        #         p2 = np.array([x[j], y[j], z[j]])
        #         dist =  np.sqrt(np.sum((p1 - p2) ** 2)) #np.linalg.norm(p1-p2)
                
        #         atom1, atom2 = idx2atom[atom_type[i].item()], idx2atom[atom_type[j].item()]
        #         bond_order = get_bond_order(atom1, atom2, dist)
                
        #         bond_order_dict[bond_order] += 1
        #         # if bond_order <= 0: #TODO debug. Why not all set to 0?
        #         if True:
        #             data["edge_mask"][i * len(x) + j] = 0
        #             data["edge_mask"][j * len(x) + i] = 0
        #         #set all edge_mask indices to 0
        #         data["edge_mask"] = torch.zeros_like(data["edge_mask"])

        #randomly mask out 50% of atoms
        # mask = torch.rand(data["atom_mask"].shape) > 0.5
        # data["atom_mask"] = data["atom_mask"] * mask.to(model.device)
        #mask out all atoms
        # data["atom_mask"] = torch.zeros_like(data["atom_mask"])
        
        #variables that will become function/class arguments/variables

        
        num_fragment_atoms = torch.sum(data["fragment_mask"] == 1)

        rng = default_rng(seed = SEED)
        phi_atoms = {}
        fragment_indices = torch.where(data["fragment_mask"] == 1)[1]
        num_fragment_atoms = len(fragment_indices)
        num_atoms = data["positions"].shape[1]

        distances_random_samples = []
        cosine_similarities_random_samples = []

        for j in tqdm(range(num_fragment_atoms)):
            
            marginal_contrib_distance = 0
            marginal_contrib_cosine_similarity = 0
            marginal_contrib_hausdorff = 0

            for step in tqdm(range(M)):
                data_j_plus = data.copy()
                data_j_minus = data.copy()
                data_random = data.copy()

                N_z_mask = rng.binomial(1, P, size = num_fragment_atoms)

                # Ensure at least one element is 1, otherwise randomly select one since at least one fragment atom must be present
                if not np.any(N_z_mask):
                    print("Zero elements in N_z_mask, randomly selecting one.")
                    random_index = rng.integers(0, num_fragment_atoms)
                    N_z_mask[random_index] = 1

                # print("N_z_mask for sample", sampled, step, N_z_mask)

                N_mask = torch.ones(num_fragment_atoms, dtype=torch.int)

                pi = torch.randperm(num_fragment_atoms)

                N_j_plus_index = torch.ones(num_fragment_atoms, dtype=torch.int)
                N_j_minus_index = torch.ones(num_fragment_atoms, dtype=torch.int)
                selected_node_index = np.where(pi == j)[0].item()
                
                # print("Selected node index", selected_node_index)
                for k in range(num_fragment_atoms):
                    if k <= selected_node_index:
                        N_j_plus_index[pi[k]] = N_mask[pi[k]]
                    else:
                        N_j_plus_index[pi[k]] = N_z_mask[pi[k]]

                for k in range(num_fragment_atoms):
                    if k < selected_node_index:
                        N_j_minus_index[pi[k]] = N_mask[pi[k]]
                    else:
                        N_j_minus_index[pi[k]] = N_z_mask[pi[k]]


                # print("N_j_plus_index", N_j_plus_index)
                # print("N_j_minus_index", N_j_minus_index)
                # print(N_j_plus_index == N_j_minus_index)
                
                N_j_plus = fragment_indices[N_j_plus_index.bool()] #fragement indices to keep in molecule j plus
                N_j_minus = fragment_indices[N_j_minus_index.bool()] #fragement indices to keep in molecule j minus

                N_random_sample = fragment_indices[torch.IntTensor(N_z_mask).bool()] #fragement indices to keep in random molecule
                # print("N_j_plus", N_j_plus)
                # print("N_j_minus", N_j_minus)
                # print(N_j_plus == N_j_minus)
                atom_mask_j_plus = torch.zeros(num_atoms, dtype=torch.bool)
                atom_mask_j_minus = torch.zeros(num_atoms, dtype=torch.bool)

                atom_mask_random_molecule = torch.zeros(num_atoms, dtype=torch.bool)

                atom_mask_j_plus[N_j_plus] = True
                #set to true also linker atoms
                atom_mask_j_plus[data["linker_mask"][0].squeeze().to(torch.int) == 1] = True
                atom_mask_j_minus[N_j_minus] = True
                #set to true also linker atoms
                atom_mask_j_minus[data["linker_mask"][0].squeeze().to(torch.int) == 1] = True

                atom_mask_random_molecule[N_random_sample] = True
                #set to true also linker atoms
                atom_mask_random_molecule[data["linker_mask"][0].squeeze().to(torch.int) == 1] = True

                # print("Atom mask j plus", atom_mask_j_plus)
                # print("Atom mask j minus", atom_mask_j_minus)
                # print(atom_mask_j_minus==atom_mask_j_plus)

                #for sample containing j
                #remove positions of atoms in random_indices
                data_j_plus["positions"] = data_j_plus["positions"][:, atom_mask_j_plus]
                #remove one_hot of atoms in random_indices
                data_j_plus["one_hot"] = data_j_plus["one_hot"][:, atom_mask_j_plus]
                #remove atom_mask of atoms in random_indices
                data_j_plus["atom_mask"] = data_j_plus["atom_mask"][:, atom_mask_j_plus]
                #remove fragment_mask of atoms in random_indices
                data_j_plus["fragment_mask"] =  data_j_plus["fragment_mask"][:, atom_mask_j_plus]
                #remove linker_mask of atoms in random_indices
                data_j_plus["linker_mask"] = data_j_plus["linker_mask"][:, atom_mask_j_plus]
                #remove edge_mask of atoms in random_indices
                for index in N_j_plus:
                    for i in range(num_atoms):
                        data_j_plus["edge_mask"][index * num_atoms + i] = 0
                        data_j_plus["edge_mask"][i * num_atoms + index] = 0

                #remove all values in edge_mask that are 0
                data_j_plus["edge_mask"] = data_j_plus["edge_mask"][data_j_plus["edge_mask"] != 0]  #to be checked, but working on atoms has as effect. For the moment we stick to atoms, then we move to edges (need to edit internal function for this, or redefine everything...)

                # print("After removal j plus:", data_j_plus["positions"])
                # print(data_j_plus["positions"].shape)
                
                #for sample not containing j
                #remove positions of atoms in random_indices
                data_j_minus["positions"] = data_j_minus["positions"][:, atom_mask_j_minus]
                #remove one_hot of atoms in random_indices
                data_j_minus["one_hot"] = data_j_minus["one_hot"][:, atom_mask_j_minus]
                #remove atom_mask of atoms in random_indices
                data_j_minus["atom_mask"] = data_j_minus["atom_mask"][:, atom_mask_j_minus]
                #remove fragment_mask of atoms in random_indices
                data_j_minus["fragment_mask"] =  data_j_minus["fragment_mask"][:, atom_mask_j_minus]
                #remove linker_mask of atoms in random_indices
                data_j_minus["linker_mask"] = data_j_minus["linker_mask"][:, atom_mask_j_minus]
                #remove edge_mask of atoms in random_indices
                for index in N_j_minus:
                    for i in range(num_atoms):
                        data_j_minus["edge_mask"][index * num_atoms + i] = 0
                        data_j_minus["edge_mask"][i * num_atoms + index] = 0

                #remove all values in edge_mask that are 0
                data_j_minus["edge_mask"] = data_j_minus["edge_mask"][data_j_minus["edge_mask"] != 0]  #to be checked, but working on atoms has as effect. For the moment we stick to atoms, then we move to edges (need to edit internal function for this, or redefine everything...)

                # print("After removal j minus:", data_j_minus["positions"])
                # print(data_j_minus["positions"].shape)

                #for random sample
                data_random["positions"] = data_random["positions"][:, atom_mask_random_molecule]
                #remove one_hot of atoms in random_indices
                data_random["one_hot"] = data_random["one_hot"][:, atom_mask_random_molecule]
                #remove atom_mask of atoms in random_indices
                data_random["atom_mask"] = data_random["atom_mask"][:, atom_mask_random_molecule]
                #remove fragment_mask of atoms in random_indices
                data_random["fragment_mask"] =  data_random["fragment_mask"][:, atom_mask_random_molecule]
                #remove linker_mask of atoms in random_indices
                data_random["linker_mask"] = data_random["linker_mask"][:, atom_mask_random_molecule]
                #remove edge_mask of atoms in random_indices
                for index in N_z_mask:
                    for i in range(num_atoms):
                        data_random["edge_mask"][index * num_atoms + i] = 0
                        data_random["edge_mask"][i * num_atoms + index] = 0

                #remove all values in edge_mask that are 0
                data_random["edge_mask"] = data_random["edge_mask"][data_random["edge_mask"] != 0] 



                #with node j
                chain_j_plus, node_mask_j_plus = model.sample_chain(data_j_plus, keep_frames=args.keep_frames)
                #take only the ts 0 frame
                chain_j_plus = chain_j_plus[0, 0, :, :]
                
            
                V_j_plus_distance = compute_molecular_distance(chain_with_full_fragments.squeeze(), chain_j_plus.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data_j_plus["linker_mask"][0].squeeze())

                V_j_plus_cosine_similarity = compute_cosine_similarity(chain_with_full_fragments.squeeze().cpu(), chain_j_plus.squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data_j_plus["linker_mask"][0].squeeze().cpu())

                # print("V_j_plus", V_j_plus)

                #without node j
                chain_j_minus, node_mask_j_minus = model.sample_chain(data_j_minus, keep_frames=args.keep_frames)

                #take only the ts 0 frame
                chain_j_minus = chain_j_minus[0, 0, :, :]

                V_j_minus_distance = compute_molecular_distance(chain_with_full_fragments.squeeze(), chain_j_minus.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data_j_minus["linker_mask"][0].squeeze())

                V_j_minus_cosine_similarity = compute_cosine_similarity(chain_with_full_fragments.squeeze().cpu(), chain_j_minus.squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data_j_minus["linker_mask"][0].squeeze().cpu())

                #with random sample
                chain_random, node_mask_random = model.sample_chain(data_random, keep_frames=args.keep_frames)

                chain_random = chain_random[0, 0, :, :]

                V_random_distance = compute_molecular_distance(chain_with_full_fragments.squeeze(), chain_random.squeeze(), mask1=data["linker_mask"][0].squeeze(), mask2=data_random["linker_mask"][0].squeeze())

                V_random_cosine_similarity = compute_cosine_similarity(chain_with_full_fragments.squeeze().cpu(), chain_random.squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data_random["linker_mask"][0].squeeze().cpu())

                distances_random_samples.append(V_random_distance)
                cosine_similarities_random_samples.append(V_random_cosine_similarity)

                # print(V_random_distance, V_random_cosine_similarity)
                
                marginal_contrib_distance += (V_j_plus_distance - V_j_minus_distance)

                marginal_contrib_cosine_similarity += (V_j_plus_cosine_similarity - V_j_minus_cosine_similarity)

                # marginal_contrib_hausdorff += (V_j_plus_hausdorff - V_j_minus_hausdorff)

            phi_atoms[fragment_indices[j].item()] = [0,0] #,0]    
            phi_atoms[fragment_indices[j].item()][0] = marginal_contrib_distance/M #j is the index of the fragment atom in the fragment indices tensor
            phi_atoms[fragment_indices[j].item()][1] = marginal_contrib_cosine_similarity/M
            # phi_atoms[fragment_indices[j]][2] = marginal_contrib_hausdorff/M

            print(data["name"])

        phi_atoms_distances = {}
        phi_atoms_cosine_similarity = {}
        for atom_index, phi_values in phi_atoms.items():
            phi_atoms_distances[atom_index] = phi_values[0]
            phi_atoms_cosine_similarity[atom_index] = phi_values[1]

        if SAVE_VISUALIZATION:
            for i in range(len(data['positions'])):
                chain = chain_batch[:, i, :, :]
                assert chain.shape[0] == args.keep_frames
                assert chain.shape[1] == data['positions'].shape[1]
                assert chain.shape[2] == data['positions'].shape[2] + data['one_hot'].shape[2] + model.include_charges

                # Saving chains
                name = str(i + start)
                chain_output = os.path.join(chains_output_dir, name)
                os.makedirs(chain_output, exist_ok=True)

                one_hot = chain[:, :, 3:-1]
                positions = chain[:, :, :3]
                chain_node_mask = torch.cat([node_mask[i].unsqueeze(0) for _ in range(args.keep_frames)], dim=0)
                names = [f'{name}_{j}' for j in range(args.keep_frames)]

                save_xyz_file(chain_output, one_hot, positions, chain_node_mask, names=names, is_geom=model.is_geom)
                visualize_chain_xai(
                    chain_output,
                    spheres_3d=True,
                    alpha=0.7,
                    bg='white',
                    is_geom=model.is_geom,
                    fragment_mask=data['fragment_mask'][i].squeeze(),
                    phi_values=phi_atoms_distances
                )

                # Saving final prediction and ground truth separately
                true_one_hot = data['one_hot'][i].unsqueeze(0)
                true_positions = data['positions'][i].unsqueeze(0)
                true_node_mask = data['atom_mask'][i].unsqueeze(0)
                save_xyz_file(
                    final_states_output_dir,
                    true_one_hot,
                    true_positions,
                    true_node_mask,
                    names=[f'{name}_true'],
                    is_geom=model.is_geom,
                )

                pred_one_hot = chain[0, :, 3:-1].unsqueeze(0)
                pred_positions = chain[0, :, :3].unsqueeze(0)
                pred_node_mask = chain_node_mask[0].unsqueeze(0)
                save_xyz_file(
                    final_states_output_dir,
                    pred_one_hot,
                    pred_positions,
                    pred_node_mask,
                    names=[f'{name}_pred'],
                    is_geom=model.is_geom
                )

            start += len(data['positions'])

        # Save phi_atoms to a text file
        with open(f'{folder_save_path}/phi_atoms_{sampled}.txt', 'w') as write_file:
            write_file.write("sample name: " + str(data["name"]) + "\n")
            write_file.write("atom_index,distance,cosine_similarity\n")
            for atom_index, phi_values in phi_atoms.items():
                write_file.write(f"{atom_index},{phi_values[0]},{phi_values[1]}\n")

            write_file.write("\n")      
            write_file.write("Distance random samples\n")
            write_file.write(str(distances_random_samples) + "\n")
            write_file.write("Cosine similarity random samples\n")
            write_file.write(str(cosine_similarities_random_samples) + "\n")
      


Similarity between the two chains: 1.0
Similarity between the two chains based on positions: 1.0
Similarity between the two one-hot vectors: 1.0
Cosine similarity between the two chains: 1.0


  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# if ATOM_SAMPLER:
#             print("Before removal:", data["positions"].shape)
#             #get all indices in atom_mask that correspond to fragment atoms
#             fragment_indices = torch.where(data["fragment_mask"] == 1)[1]

#             print(fragment_indices)
            
#             #randomly pick 50% of fragment atoms
#             random_indices = torch.randperm(len(fragment_indices))[:int(len(fragment_indices)/2)]
#             mask_fragments = torch.ones(len(fragment_indices), dtype=torch.bool)
#             mask_fragments[random_indices] = False
#             selected_fragment_atoms = fragment_indices[mask_fragments]

#             #keep only the first index in selected_fragment_atoms
#             selected_fragment_atoms = selected_fragment_atoms[:1] #@mastro change this removing the explained atom
#             print("Selected fragment atoms", selected_fragment_atoms)
            
#             num_atoms = data["positions"].shape[1]
#             # random_indices = torch.randperm(num_atoms)[:int(num_atoms/2)]
#             mask = torch.ones(num_atoms, dtype=torch.bool)
#             mask[selected_fragment_atoms] = False


#             #remove positions of atoms in random_indices
#             data["positions"] = data["positions"][:, mask]
#             #remove one_hot of atoms in random_indices
#             data["one_hot"] = data["one_hot"][:, mask]
#             #remove atom_mask of atoms in random_indices
#             data["atom_mask"] = data["atom_mask"][:, mask]
#             #remove fragment_mask of atoms in random_indices
#             data["fragment_mask"] =  data["fragment_mask"][:, mask]
#             #remove linker_mask of atoms in random_indices
#             data["linker_mask"] = data["linker_mask"][:, mask]
#             #remove edge_mask of atoms in random_indices
#             for index in random_indices:
#                 for i in range(num_atoms):
#                     data["edge_mask"][index * num_atoms + i] = 0
#                     data["edge_mask"][i * num_atoms + index] = 0

#             #remove all values in edge_mask that are 0
#             data["edge_mask"] = data["edge_mask"][data["edge_mask"] != 0]  #to be checked, but working on atoms has as effect. For the moment we stick to atoms, then we move to edges (need to edit internal function for this, or redefine everything...)
            

#             print("After removal:", data["positions"].shape)
#             # sys.exit()
#             # print number of zeros in edge mask
#             print("Number of masked out edges (edges not representing bonds)", torch.sum(data["edge_mask"] == 0))
#             print("Number of edges still present", torch.sum(data["edge_mask"] != 0))

#             # print number of zeros in atom mask
#             print("Number of masked out atoms", torch.sum(data["atom_mask"] == 0))

## Visualization - trials

Implementing visualization highlighting important atoms

In [ ]:
# phi_atoms = {0: -0.1030,
#  1: -2.5306,
#  2: 0.3042,
#  3: -1.4719,
#  4: -1.3421,
#  5: -0.2840,
#  6: 0.9049,
#  7: -1.1589,
#  8: -0.2158,
#  9: -1.8067,
#  10: -2.1304,
#  11: -0.2276,
#  12: -1.6576,
#  13: -0.0887,
#  14: 0.0418,
#  15: -1.8112,
#  16: 0.4877,
#  17: 0.0786,
#  18: -0.2415,
#  19: -1.5389,
#  20: -1.8475}

# shapley_values = {0:(-0.890280568599701,0.06123639730736613),
# 1:(0.18284012019634246,-0.0529984375834465),
# 2:(-4.252275815010071,0.061238834373652934),
# 3:(-2.081632282733917,0.26110127678141),
# 4:(0.01699479103088379,-0.013148795943707228),
# 5:(-0.24424300074577332,0.044520441107451916),
# 6:(0.5019020104408264,-0.06629922293126583),
# 7:(0.39756537079811094,-0.04101886671036482),
# 8:(0.2415522527694702,0.0025899429991841317),
# 9:(-1.1490421509742736,0.17777293106541037),
# 10:(-1.3627109980583192,0.22668330844491721),
# 11:(-0.17836040258407593,0.03489074512384832),
# 12:(-1.182363440990448,0.11668485717847943),
# 13:(0.34586033344268796,-0.02710286658257246),
# 14:(-3.1875360774993897,0.04069803573191166),
# 15:(-0.07272281646728515,0.019701205156743527),
# 16:(-0.9446532082557678,-0.05654389940202236),
# 17:(-4.32677052974701,0.10536605073139071),
# 18:(-0.32534114360809324,-0.0567812330275774),
# 19:(-0.345893794298172,0.029798283874988556),
# 20:(0.19235227584838868,-0.014152660053223371)}

# phi_atoms_distances = {}
# phi_atoms_cosine_similarity = {}

# for atom_index, phi_values in shapley_values.items():
#     phi_atoms_distances[atom_index] = phi_values[0]
#     phi_atoms_cosine_similarity[atom_index] = phi_values[1]

# print(phi_atoms_distances)
# print(phi_atoms_cosine_similarity)

{0: -0.890280568599701, 1: 0.18284012019634246, 2: -4.252275815010071, 3: -2.081632282733917, 4: 0.01699479103088379, 5: -0.24424300074577332, 6: 0.5019020104408264, 7: 0.39756537079811094, 8: 0.2415522527694702, 9: -1.1490421509742736, 10: -1.3627109980583192, 11: -0.17836040258407593, 12: -1.182363440990448, 13: 0.34586033344268796, 14: -3.1875360774993897, 15: -0.07272281646728515, 16: -0.9446532082557678, 17: -4.32677052974701, 18: -0.32534114360809324, 19: -0.345893794298172, 20: 0.19235227584838868}
{0: 0.06123639730736613, 1: -0.0529984375834465, 2: 0.061238834373652934, 3: 0.26110127678141, 4: -0.013148795943707228, 5: 0.044520441107451916, 6: -0.06629922293126583, 7: -0.04101886671036482, 8: 0.0025899429991841317, 9: 0.17777293106541037, 10: 0.22668330844491721, 11: 0.03489074512384832, 12: 0.11668485717847943, 13: -0.02710286658257246, 14: 0.04069803573191166, 15: 0.019701205156743527, 16: -0.05654389940202236, 17: 0.10536605073139071, 18: -0.0567812330275774, 19: 0.029798283

#### Utility functions for visualization modified from DiffLinker work

In [ ]:
# phi_values_array = np.array(list(phi_atoms.values()))
# phi_values_array.shape

(21,)

### Visualization step

In [ ]:
# start = 0
# for data in tqdm(dataloader):
#     chain_batch, node_mask = model.sample_chain(data, keep_frames=args.keep_frames)
#     for i in tqdm(range(len(data['positions']))):
#         chain = chain_batch[:, i, :, :]
#         assert chain.shape[0] == args.keep_frames
#         assert chain.shape[1] == data['positions'].shape[1]
#         assert chain.shape[2] == data['positions'].shape[2] + data['one_hot'].shape[2] + model.include_charges

#         # Saving chains
#         name = str(i + start)
#         chain_output = os.path.join(chains_output_dir, name)
#         os.makedirs(chain_output, exist_ok=True)

#         one_hot = chain[:, :, 3:-1]
#         positions = chain[:, :, :3]
#         chain_node_mask = torch.cat([node_mask[i].unsqueeze(0) for _ in range(args.keep_frames)], dim=0)
#         names = [f'{name}_{j}' for j in range(args.keep_frames)]

#         save_xyz_file(chain_output, one_hot, positions, chain_node_mask, names=names, is_geom=model.is_geom)
#         visualize_chain_xai(
#             chain_output,
#             spheres_3d=False,
#             alpha=0.7,
#             bg='white',
#             is_geom=model.is_geom,
#             fragment_mask=data['fragment_mask'][i].squeeze(),
#             phi_values=phi_atoms_cosine_similarity
#         )

#         # Saving final prediction and ground truth separately
#         true_one_hot = data['one_hot'][i].unsqueeze(0)
#         true_positions = data['positions'][i].unsqueeze(0)
#         true_node_mask = data['atom_mask'][i].unsqueeze(0)
#         save_xyz_file(
#             final_states_output_dir,
#             true_one_hot,
#             true_positions,
#             true_node_mask,
#             names=[f'{name}_true'],
#             is_geom=model.is_geom,
#         )

#         pred_one_hot = chain[0, :, 3:-1].unsqueeze(0)
#         pred_positions = chain[0, :, :3].unsqueeze(0)
#         pred_node_mask = chain_node_mask[0].unsqueeze(0)
#         save_xyz_file(
#             final_states_output_dir,
#             pred_one_hot,
#             pred_positions,
#             pred_node_mask,
#             names=[f'{name}_pred'],
#             is_geom=model.is_geom
#         )

#     start += len(data['positions'])
#     break

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Mastro\AppData\Local\Temp\ipykernel_21748\1457623096.py:203: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [imageio.imread(fn) for fn in save_paths]


In [ ]:
# import numpy as np

# chain_with_full_fragments = chain_with_full_fragments[0,:,:]
# # Generate a random boolean mask
# mask = np.random.choice([True, False], size=chain_with_full_fragments.shape[0])

# # Apply the mask to remove random rows
# masked_chain = chain_with_full_fragments[mask, :]

# masked_chain.shape

In [ ]:
# for data in dataloader:
#     positions = data["positions"]
#     # print(positions)
    
#     cos_sim = cosine_similarity((positions[0].squeeze().detach().cpu().numpy().flatten().reshape(1, -1)),positions[0].squeeze().detach().cpu().numpy().flatten().reshape(1, -1))

#     cosine = compute_cosine_similarity(positions[0].squeeze().cpu(), positions[0].squeeze().cpu(), mask1=data["linker_mask"][0].squeeze().cpu(), mask2=data["linker_mask"][0].squeeze().cpu())
#     print(cosine)
#     break

In [ ]:
# atom_mask_random_molecule

## Checking Shapley value Propeties

In [ ]:
# shapley_values = {0:(-0.18252010345458985,-0.03153111279010773),
# 1:(-0.19735857963562012,-0.0004155013896524906),
# 2:(0.24352870702743531,0.03291264953091741),
# 3:(0.5766246366500855,-0.02155731648206711),
# 4:(-0.8824016571044921,-0.02419700352475047),
# 5:(0.04895777225494385,-0.02288945931941271),
# 6:(-0.11883691549301148,-0.017148097790777684),
# 7:(-0.3973711347579956,-0.08772553377784789),
# 8:(-1.0809556245803833,0.005452861245721578),
# 9:(-0.09876126766204835,-0.04913015581667423),
# 10:(-0.9884893560409546,0.11794438790529967),
# 11:(-1.126043050289154,0.13286019276827574),
# 12:(-1.1925089359283447,-0.07200432924553751),
# 13:(-1.183656153678894,0.11058532498776913),
# 14:(-1.2386692070960998,0.12144924929365515),
# 15:(-0.9519470238685608,0.09354953311383724),
# 16:(-1.0259105682373046,0.1189951341226697),
# 17:(-0.47114646434783936,-0.006474981680512428),
# 18:(-0.516231164932251,0.08100643368437886),
# 19:(-1.2924485397338867,0.13028908021748065)}


In [ ]:
# sum_shapley_values_distance = 0
# sum_shapley_values_cosine_similarity = 0

# for key, value in shapley_values.items():
#     sum_shapley_values_distance += value[0]
#     sum_shapley_values_cosine_similarity += value[1]

# print("Sum of shapley values for distance", sum_shapley_values_distance)
# print("Sum of shapley values for cosine similarity", sum_shapley_values_cosine_similarity)

Sum of shapley values for distance -12.076144630908965
Sum of shapley values for cosine similarity 0.6119713550526649


In [ ]:
distances_random_graphs = [18.90079116821289, 13.994956016540527, 18.59467315673828, 99.57367706298828, 12.549628257751465, 12.131684303283691, 33.354164123535156, 12.624557495117188, 7.622239589691162, 7.140101909637451, 8.129176139831543, 11.115296363830566, 11.150999069213867, 12.553301811218262, 18.150859832763672, 39.439090728759766, 14.725622177124023, 12.986281394958496, 9.173707962036133, 11.731589317321777, 10.623479843139648, 14.190513610839844, 9.719880104064941, 22.595802307128906, 130.5957489013672, 11.12348747253418, 16.891321182250977, 15.867040634155273, 6.8220624923706055, 15.32767391204834, 14.361724853515625, 13.085883140563965, 9.035792350769043, 11.632275581359863, 20.091903686523438, 15.532211303710938, 117.51826477050781, 12.189861297607422, 16.283920288085938, 18.390846252441406, 8.840758323669434, 15.95419692993164, 9.660080909729004, 8.623753547668457, 17.921588897705078, 12.339995384216309, 10.192910194396973, 14.382380485534668, 17.58751678466797, 8.23885440826416, 13.433709144592285, 9.593979835510254, 10.967704772949219, 12.773087501525879, 17.316118240356445, 16.888185501098633, 10.666703224182129, 9.54337215423584, 13.131654739379883, 8.862771987915039, 20.0909423828125, 12.038680076599121, 8.496023178100586, 15.084773063659668, 28.776079177856445, 14.874223709106445, 9.538623809814453, 8.145123481750488, 17.69808006286621, 11.451218605041504, 17.596590042114258, 7.713534832000732, 9.267410278320312, 8.377812385559082, 12.443464279174805, 9.23165225982666, 7.949616432189941, 14.71863079071045, 11.06482982635498, 18.3727970123291, 19.899072647094727, 12.815223693847656, 9.274666786193848, 15.73747444152832, 13.252983093261719, 8.710273742675781, 8.975485801696777, 15.580423355102539, 11.826573371887207, 14.264101028442383, 31.91473960876465, 64.17339324951172, 17.470298767089844, 8.343291282653809, 137.162353515625, 17.53420066833496, 122.32755279541016, 13.094176292419434, 12.624659538269043, 13.29981517791748, 92.17593383789062, 16.668046951293945, 17.38125991821289, 15.251635551452637, 14.045061111450195, 12.001233100891113, 19.510133743286133, 17.609756469726562, 11.652176856994629, 10.514715194702148, 14.88312816619873, 13.046287536621094, 70.30615234375, 8.78591251373291, 8.672098159790039, 16.624267578125, 11.7927827835083, 10.962799072265625, 10.831814765930176, 8.06410026550293, 9.344090461730957, 10.921401023864746, 148.69432067871094, 13.781452178955078, 13.000008583068848, 13.64437198638916, 14.806971549987793, 64.73077392578125, 17.488983154296875, 10.001614570617676, 13.821581840515137, 9.20461368560791, 18.423873901367188, 11.150386810302734, 13.311408996582031, 8.561570167541504, 69.8349609375, 16.09000587463379, 10.647528648376465, 12.956040382385254, 21.508037567138672, 14.847746849060059, 15.939468383789062, 211.32839965820312, 8.488847732543945, 13.12687873840332, 27.037609100341797, 16.800561904907227, 16.64445686340332, 13.937997817993164, 19.06344223022461, 19.3958797454834, 8.182333946228027, 15.081774711608887, 13.452703475952148, 9.053831100463867, 10.179545402526855, 11.762200355529785, 15.534356117248535, 14.996820449829102, 17.824600219726562, 11.682884216308594, 13.216797828674316, 13.14797306060791, 88.205322265625, 13.995166778564453, 13.644662857055664, 126.34536743164062, 13.231528282165527, 8.826067924499512, 8.81607437133789, 12.175833702087402, 12.967278480529785, 11.300259590148926, 12.155691146850586, 8.25037670135498, 10.764246940612793, 11.88794231414795, 14.512004852294922, 12.95032024383545, 13.645676612854004, 11.74242115020752, 11.460274696350098, 14.68875503540039, 14.583455085754395, 13.928496360778809, 8.401372909545898, 9.419363975524902, 15.985710144042969, 9.360016822814941, 47.873226165771484, 158.76870727539062, 12.506614685058594, 15.970625877380371, 17.951297760009766, 12.506709098815918, 12.172075271606445, 10.580382347106934, 58.27928161621094, 15.034242630004883, 50.50627136230469, 15.334737777709961, 15.018738746643066, 160.43295288085938, 8.316046714782715, 8.562100410461426, 15.030044555664062, 71.23239135742188, 15.056232452392578, 13.78482437133789, 14.468690872192383, 13.743317604064941, 11.187176704406738, 6.418200492858887, 18.091672897338867, 18.416152954101562, 10.001916885375977, 15.015134811401367, 22.67055320739746, 21.574569702148438, 15.522256851196289, 8.886856079101562, 14.800872802734375, 9.598064422607422, 11.920475959777832, 142.5004119873047, 61.655277252197266, 11.745392799377441, 13.361960411071777, 12.651230812072754, 15.173436164855957, 10.20767593383789, 11.558145523071289, 13.817797660827637, 17.00383758544922, 86.9465103149414, 14.227371215820312, 8.502904891967773, 63.643775939941406, 16.2381649017334, 68.77086639404297, 16.842241287231445, 29.20728302001953, 104.66841888427734, 13.297395706176758, 11.490921974182129, 11.665875434875488, 15.498967170715332, 15.983936309814453, 15.490246772766113, 94.63960266113281, 12.658391952514648, 12.584333419799805, 61.490421295166016, 13.70885944366455, 9.461601257324219, 16.171279907226562, 14.173911094665527, 15.558684349060059, 8.964704513549805, 12.975093841552734, 38.12180709838867, 14.698602676391602, 12.251805305480957, 8.364845275878906, 73.21941375732422, 12.468559265136719, 10.806327819824219, 15.765046119689941, 38.61955642700195, 25.2303466796875, 10.55815601348877, 16.790437698364258, 16.72580337524414, 10.947154998779297, 44.476951599121094, 11.342514038085938, 9.823236465454102, 13.439870834350586, 14.057900428771973, 15.702348709106445, 123.84170532226562, 10.194940567016602, 11.569419860839844, 16.279428482055664, 11.077016830444336, 17.648103713989258, 10.986272811889648, 11.707524299621582, 13.497894287109375, 114.54523468017578, 60.219261169433594, 11.34465503692627, 180.37362670898438, 17.984201431274414, 10.890538215637207, 8.185347557067871, 175.22923278808594, 41.305606842041016, 16.288894653320312, 10.930126190185547, 9.697709083557129, 9.454169273376465, 16.14569091796875, 7.287662029266357, 19.164567947387695, 158.73419189453125, 8.18952751159668, 12.114006996154785, 14.718876838684082, 124.69756317138672, 13.660598754882812, 7.137762546539307, 14.310454368591309, 13.83088207244873, 13.601731300354004, 11.374911308288574, 12.588821411132812, 10.643749237060547, 14.501836776733398, 7.655153751373291, 25.34112548828125, 16.27726936340332, 12.277924537658691, 13.433985710144043, 15.98641586303711, 11.581562042236328, 15.882213592529297, 15.093315124511719, 10.40549087524414, 10.727749824523926, 69.3736343383789, 12.763786315917969, 6.882559299468994, 8.245793342590332, 8.511961936950684, 13.76906681060791, 14.810815811157227, 14.240970611572266, 73.23014068603516, 8.3338041305542, 8.692272186279297, 16.71268653869629, 76.13397979736328, 11.38884449005127, 15.482198715209961, 13.273999214172363, 13.864989280700684, 14.873936653137207, 15.938603401184082, 15.499252319335938, 9.533032417297363, 12.131134033203125, 14.583467483520508, 10.751606941223145, 12.345020294189453, 9.381270408630371, 12.66125202178955, 8.552224159240723, 33.45722579956055, 12.638304710388184, 95.25855255126953, 12.710970878601074, 8.803593635559082, 8.221055030822754, 15.017343521118164, 9.953560829162598, 52.0971794128418, 18.072296142578125, 7.914463520050049, 14.725645065307617, 13.39150333404541, 11.575958251953125, 13.230031967163086, 9.491990089416504, 12.860107421875, 7.974977016448975, 10.814910888671875, 10.971990585327148, 11.948029518127441, 15.91380786895752, 11.850900650024414, 30.33868408203125, 16.38983726501465, 13.10588264465332, 81.3344955444336, 14.5990571975708, 7.989089488983154, 12.68564510345459, 11.342852592468262, 9.488457679748535, 17.77791976928711, 13.5999174118042, 37.581878662109375, 10.817098617553711, 16.679418563842773, 52.27433395385742, 8.068964958190918, 16.101106643676758, 15.701685905456543, 10.257023811340332, 14.741579055786133, 14.575613021850586, 15.70040225982666, 14.47760009765625, 11.149016380310059, 15.399667739868164, 12.463533401489258, 12.344133377075195, 9.85230827331543, 10.088934898376465, 16.35111427307129, 11.112174987792969, 16.054683685302734, 21.263277053833008, 16.61769676208496, 12.683947563171387, 11.60739517211914, 13.781305313110352, 44.46507263183594, 12.36363697052002, 17.617530822753906, 11.271045684814453, 15.722209930419922, 11.535789489746094, 12.749752044677734, 7.245047092437744, 25.049476623535156, 9.355659484863281, 91.5381088256836, 13.09835147857666, 43.77478790283203, 13.353759765625, 11.243021965026855, 14.344939231872559, 160.9538116455078, 22.586719512939453, 61.63052749633789, 11.218603134155273, 14.148180961608887, 12.577577590942383, 16.64189910888672, 9.968767166137695, 13.764602661132812, 92.05048370361328, 16.939903259277344, 6.361188888549805, 132.25189208984375, 13.822798728942871, 17.46392822265625, 15.206936836242676, 15.252385139465332, 11.729472160339355, 9.917499542236328, 14.209548950195312, 8.2781982421875, 18.495769500732422, 14.957857131958008, 13.36463737487793, 67.4285888671875, 11.830961227416992, 15.976922035217285, 10.101573944091797, 9.5398530960083, 9.283186912536621, 16.630590438842773, 82.12898254394531, 10.795676231384277, 9.574674606323242, 8.211753845214844, 8.317020416259766, 144.6425018310547, 17.35848617553711, 19.046621322631836, 10.673645973205566, 17.231094360351562, 14.196640014648438, 15.906322479248047, 95.96017456054688, 16.897001266479492, 15.440664291381836, 11.983726501464844, 6.6015543937683105, 13.456393241882324, 16.277090072631836, 8.065096855163574, 12.918499946594238, 10.775928497314453, 146.80747985839844, 122.9027099609375, 13.884522438049316, 7.073297023773193, 121.78899383544922, 17.756973266601562, 11.461078643798828, 14.569354057312012, 10.598976135253906, 15.125158309936523, 14.1181058883667, 17.792455673217773, 19.60222625732422, 11.961310386657715, 15.116540908813477, 16.01392364501953, 11.030943870544434, 8.910484313964844, 9.509554862976074, 17.800067901611328, 15.164569854736328, 14.670177459716797, 88.6915054321289, 121.71558380126953, 12.757003784179688, 14.310590744018555, 9.07774543762207, 16.862485885620117, 9.439908027648926, 10.220917701721191, 154.8267364501953, 14.240350723266602, 40.20277786254883, 20.16124725341797, 16.219444274902344, 16.65301513671875, 10.62325668334961, 13.734065055847168, 15.613786697387695, 8.161663055419922, 11.33582878112793, 8.462597846984863, 9.912310600280762, 14.480301856994629, 8.693795204162598, 45.62960433959961, 11.716885566711426, 10.923089981079102, 17.40431785583496, 17.74018096923828, 12.137772560119629, 13.51106071472168, 11.30299186706543, 164.84873962402344, 12.02062702178955, 15.675127029418945, 15.032716751098633, 9.049527168273926, 12.351417541503906, 14.299474716186523, 14.794717788696289, 12.363577842712402, 16.672121047973633, 11.58381462097168, 12.191515922546387, 14.990013122558594, 16.577877044677734, 76.04591369628906, 9.153544425964355, 17.500690460205078, 8.538971900939941, 127.1612548828125, 25.207828521728516, 11.293585777282715, 15.654316902160645, 86.48731994628906, 9.346173286437988, 14.346590042114258, 11.479499816894531, 10.730838775634766, 16.054508209228516, 15.093586921691895, 19.79429817199707, 10.897029876708984, 17.370534896850586, 14.462530136108398, 11.198225021362305, 9.267816543579102, 10.98493766784668, 11.289857864379883, 17.239336013793945, 13.556992530822754, 41.982601165771484, 9.476637840270996, 13.523673057556152, 18.27334213256836, 11.9056396484375, 7.497525691986084, 9.317440032958984, 11.975285530090332, 27.088430404663086, 131.57476806640625, 12.991883277893066, 15.491015434265137, 7.88443660736084, 12.811760902404785, 12.648347854614258, 17.135892868041992, 13.93150806427002, 20.809669494628906, 34.16574478149414, 7.8645405769348145, 11.909616470336914, 10.309382438659668, 10.354181289672852, 15.613704681396484, 36.93091583251953, 110.46654510498047, 9.712177276611328, 8.263484954833984, 9.402942657470703, 125.7695541381836, 13.111052513122559, 10.161175727844238, 8.154107093811035, 16.999860763549805, 15.889485359191895, 15.779815673828125, 122.34123229980469, 9.410789489746094, 15.567009925842285, 142.2126007080078, 9.809372901916504, 12.531021118164062, 14.250614166259766, 17.9440975189209, 11.193950653076172, 13.94561767578125, 17.39723014831543, 14.717508316040039, 11.178109169006348, 11.863083839416504, 13.983922958374023, 16.899805068969727, 17.37537384033203, 50.51288604736328, 90.2881851196289, 114.02394104003906, 9.9369478225708, 14.581583976745605, 115.93660736083984, 91.25593566894531, 17.302541732788086, 16.615230560302734, 10.671255111694336, 14.109707832336426, 15.769636154174805, 8.390755653381348, 11.630675315856934, 11.566071510314941, 12.012032508850098, 13.529691696166992, 15.081244468688965, 7.619426250457764, 10.07054328918457, 11.75484561920166, 8.847921371459961, 18.49893569946289, 12.589421272277832, 11.385807991027832, 13.44986629486084, 7.3566789627075195, 11.126753807067871, 115.1276626586914, 11.853428840637207, 9.208089828491211, 12.032096862792969, 61.5062255859375, 64.83589172363281, 12.81176471710205, 15.652209281921387, 11.118395805358887, 8.029684066772461, 8.396458625793457, 68.54464721679688, 169.66842651367188, 12.024041175842285, 10.978395462036133, 9.908374786376953, 16.47556495666504, 10.23449993133545, 9.79993724822998, 10.829285621643066, 11.360042572021484, 64.6798324584961, 9.917633056640625, 106.91581726074219, 21.220693588256836, 15.75705623626709, 7.881631374359131, 9.127352714538574, 12.71451473236084, 8.411365509033203, 21.13786506652832, 10.872757911682129, 16.53838539123535, 12.896525382995605, 12.709114074707031, 12.073543548583984, 14.947966575622559, 8.916463851928711, 16.108610153198242, 10.707052230834961, 8.583785057067871, 10.001741409301758, 188.49835205078125, 12.740950584411621, 10.625408172607422, 12.930603981018066, 17.968597412109375, 8.518970489501953, 9.465893745422363, 16.741853713989258, 8.966487884521484, 12.891875267028809, 10.23681640625, 14.482185363769531, 12.775634765625, 21.32612419128418, 15.98818588256836, 12.199485778808594, 14.404911994934082, 16.117820739746094, 30.77677345275879, 123.97537231445312, 10.415335655212402, 7.750854015350342, 12.815423011779785, 14.577651977539062, 8.151029586791992, 18.81167221069336, 13.661920547485352, 12.445822715759277, 11.601381301879883, 18.45810317993164, 12.547820091247559, 8.371438980102539, 42.407291412353516, 148.1404571533203, 8.784523963928223, 11.032090187072754, 10.364784240722656, 13.131935119628906, 12.029441833496094, 134.8007049560547, 15.829763412475586, 12.772907257080078, 85.35541534423828, 14.088205337524414, 15.86083698272705, 9.73794937133789, 6.406563758850098, 7.324241638183594, 10.341604232788086, 10.299869537353516, 10.412753105163574, 12.475834846496582, 13.548815727233887, 54.29713821411133, 16.655685424804688, 13.365704536437988, 9.260379791259766, 162.299560546875, 12.094717979431152, 11.122003555297852, 8.614922523498535, 12.408309936523438, 21.27689552307129, 8.919564247131348, 14.129417419433594, 14.39644718170166, 9.93635368347168, 17.97496795654297, 13.67654037475586, 9.598725318908691, 15.049284934997559, 13.420178413391113, 17.3514404296875, 11.059821128845215, 16.323352813720703, 10.138786315917969, 18.3267822265625, 10.944792747497559, 15.471132278442383, 13.489191055297852, 14.863912582397461, 10.311471939086914, 17.876075744628906, 9.679759979248047, 77.70812225341797, 8.807772636413574, 8.622052192687988, 15.812366485595703, 16.786279678344727, 8.209694862365723, 8.524859428405762, 16.198617935180664, 13.111818313598633, 17.490846633911133, 42.85334396362305, 9.830799102783203, 18.8314266204834, 13.832694053649902, 12.781755447387695, 14.963619232177734, 12.25667667388916, 15.194893836975098, 13.423351287841797, 18.47365951538086, 15.174413681030273, 13.024310111999512, 31.7904109954834, 10.487093925476074, 15.917614936828613, 9.426745414733887, 14.892796516418457, 148.93496704101562, 9.716269493103027, 20.793682098388672, 13.964791297912598, 11.25097370147705, 8.751522064208984, 39.67639923095703, 9.051158905029297, 10.735374450683594, 14.896947860717773, 15.967572212219238, 9.30162525177002, 10.059508323669434, 14.035954475402832, 8.730210304260254, 15.771181106567383, 11.526229858398438, 7.970870494842529, 13.691450119018555, 13.06148910522461, 12.446720123291016, 204.30540466308594, 13.17358684539795, 10.232640266418457, 9.17026424407959, 88.32855224609375, 12.93061351776123, 12.7854585647583, 12.096210479736328, 49.286834716796875, 15.106887817382812, 50.01146697998047, 17.92214584350586, 7.609434127807617, 10.46533203125, 18.008625030517578, 81.21482849121094, 17.028583526611328, 14.46327018737793, 19.01139259338379, 8.826202392578125, 9.07180118560791, 14.622529983520508, 8.380635261535645, 55.35602569580078, 8.258883476257324, 16.29073143005371, 10.455572128295898, 20.127586364746094, 13.133329391479492, 12.48072338104248, 12.868857383728027, 16.03815460205078, 13.329858779907227, 15.7769136428833, 15.428328514099121, 11.135578155517578, 13.181337356567383, 12.240649223327637, 75.84722137451172, 11.936358451843262, 15.444389343261719, 14.233091354370117, 11.60185432434082, 9.690330505371094, 14.526843070983887, 13.080111503601074, 7.118505477905273, 89.80426025390625, 12.168790817260742, 12.483967781066895, 15.006213188171387, 9.729013442993164, 9.72101879119873, 12.523168563842773, 10.507203102111816, 7.595111846923828, 77.05477142333984, 9.424782752990723, 18.071924209594727, 11.77208137512207, 10.902196884155273, 10.900819778442383, 12.065722465515137, 8.634777069091797, 11.419515609741211, 8.845199584960938, 10.086554527282715, 15.793200492858887, 10.966211318969727, 70.32830047607422, 13.865107536315918, 10.069563865661621, 10.509629249572754, 17.530712127685547, 12.491271018981934, 11.684494972229004, 111.14961242675781, 21.712448120117188, 20.153608322143555, 11.768848419189453, 7.244041919708252, 11.688608169555664, 113.57896423339844, 10.583222389221191, 14.719371795654297, 12.471319198608398, 13.271402359008789, 16.69657325744629, 10.362616539001465, 13.616548538208008, 16.907411575317383, 10.382896423339844, 14.459882736206055, 10.56044864654541, 108.70511627197266, 17.63241958618164, 9.403162956237793, 10.517181396484375, 16.527746200561523, 10.745828628540039, 17.336017608642578, 15.391510963439941, 21.459474563598633, 12.48100757598877, 18.437801361083984, 10.490355491638184, 15.3898286819458, 7.874648094177246, 165.45828247070312, 18.88911247253418, 12.407785415649414, 13.909478187561035, 8.247156143188477, 76.63086700439453, 12.84585189819336, 8.043312072753906, 17.522844314575195, 17.319557189941406, 7.127760887145996, 69.98431396484375, 12.52329158782959, 11.43929672241211, 13.27674674987793, 13.405800819396973, 12.074376106262207, 13.440492630004883, 11.78515911102295, 74.97896575927734, 68.62285614013672, 10.852703094482422, 17.204923629760742, 14.232257843017578, 14.080656051635742, 12.955343246459961, 17.347497940063477, 9.534408569335938, 15.437414169311523, 16.932207107543945, 9.786357879638672, 9.494029998779297, 12.545324325561523, 10.179048538208008, 15.592337608337402, 6.58146333694458, 9.287921905517578, 12.201956748962402, 9.861615180969238, 14.916272163391113, 9.806845664978027, 82.7476806640625, 14.882879257202148, 12.183152198791504, 12.247210502624512, 133.39393615722656, 9.964488983154297, 11.048604965209961, 17.309999465942383, 14.39520263671875, 9.582695007324219, 12.580489158630371, 6.145740509033203, 11.07127571105957, 8.488987922668457, 9.356037139892578, 13.377341270446777, 8.842772483825684, 36.42991256713867, 11.115800857543945, 14.033597946166992, 9.690781593322754, 9.241148948669434, 17.303897857666016, 16.15372085571289, 15.520454406738281, 49.242881774902344, 12.242438316345215, 11.96880054473877, 10.111326217651367, 86.84037017822266, 9.087281227111816, 11.062882423400879, 13.436206817626953, 16.39241600036621, 11.22412109375, 17.93040657043457, 16.474069595336914, 13.448121070861816, 15.356689453125, 13.23005485534668, 8.461274147033691, 10.107560157775879, 13.353038787841797, 12.003473281860352, 145.40252685546875, 15.766562461853027, 8.282756805419922, 13.66427993774414, 12.264433860778809, 10.587611198425293, 11.153851509094238, 17.458417892456055, 10.502360343933105, 10.300375938415527, 15.748294830322266, 13.437994956970215, 9.726634979248047, 15.509413719177246, 119.28839874267578, 13.14885139465332, 116.15628814697266, 13.01545238494873, 7.928047180175781, 17.241518020629883, 9.629203796386719, 15.631152153015137, 11.920665740966797, 13.948360443115234, 10.70836353302002, 13.750730514526367, 8.176996231079102, 13.222097396850586, 14.296723365783691, 14.386137962341309, 10.455567359924316, 9.368335723876953, 14.414844512939453, 200.73892211914062, 12.650993347167969, 8.932271003723145, 18.37358856201172, 17.155948638916016, 8.568524360656738, 11.02763557434082, 10.128899574279785, 9.491814613342285, 16.615150451660156, 10.486967086791992, 13.00468635559082, 11.738430976867676, 8.185203552246094, 12.427055358886719, 54.96487045288086, 13.918388366699219, 11.714754104614258, 18.343170166015625, 18.632213592529297, 16.297292709350586, 14.442514419555664, 14.25227165222168, 7.634289741516113, 16.552425384521484, 8.804688453674316, 12.513901710510254, 15.045384407043457, 12.054466247558594, 14.841218948364258, 9.549818992614746, 11.6065673828125, 9.585342407226562, 17.925939559936523, 14.136693000793457, 72.39817810058594, 11.415637016296387, 79.70830535888672, 135.14144897460938, 9.38552474975586, 9.728254318237305, 12.00790786743164, 18.16031265258789, 7.6241230964660645, 9.524994850158691, 9.994375228881836, 15.370379447937012, 18.831188201904297, 10.72928237915039, 112.65670013427734, 10.55886173248291, 10.058725357055664, 9.769598007202148, 7.9361348152160645, 10.99619197845459, 11.654561042785645, 12.293506622314453, 13.608602523803711, 11.500176429748535, 10.815601348876953, 12.306047439575195, 16.52411651611328, 18.36687660217285, 134.11083984375, 13.182328224182129, 11.688969612121582, 17.090534210205078, 11.653085708618164, 17.667102813720703, 45.08943176269531, 11.00928020477295, 8.088905334472656, 13.244832038879395, 13.003801345825195, 11.862648010253906, 12.640564918518066, 11.748431205749512, 13.812063217163086, 13.873429298400879, 15.5785551071167, 10.269407272338867, 8.414214134216309, 15.981781959533691, 11.566768646240234, 10.974544525146484, 28.314725875854492, 18.977216720581055, 7.211114406585693, 15.308700561523438, 11.790360450744629, 14.743791580200195, 16.812164306640625, 10.188529968261719, 58.827327728271484, 17.464954376220703, 22.578125, 18.58675765991211, 17.71227264404297, 6.98926305770874, 15.48910140991211, 8.024348258972168, 15.987847328186035, 13.585794448852539, 17.539188385009766, 9.082531929016113, 17.1212215423584, 11.073295593261719, 16.477598190307617, 17.57328987121582, 9.670476913452148, 13.493156433105469, 17.931528091430664, 12.7913236618042, 8.605384826660156, 10.859980583190918, 11.84691333770752, 8.431897163391113, 7.553295135498047, 16.281810760498047, 10.678600311279297, 9.720598220825195, 12.035465240478516, 8.292354583740234, 121.91763305664062, 11.85035228729248, 10.544047355651855, 14.646819114685059, 98.68819427490234, 89.7779312133789, 14.242366790771484, 111.6526870727539, 17.845874786376953, 14.593854904174805, 9.850015640258789, 17.44451332092285, 11.412683486938477, 17.005123138427734, 10.061537742614746, 87.99598693847656, 6.843605995178223, 15.765785217285156, 9.148211479187012, 109.92838287353516, 8.573640823364258, 12.525510787963867, 13.185657501220703, 7.755990028381348, 13.95814323425293, 13.868263244628906, 12.790409088134766, 84.53160095214844, 12.606728553771973, 14.464646339416504, 15.226615905761719, 13.693001747131348, 8.811415672302246, 10.050363540649414, 46.7839469909668, 11.964158058166504, 18.41653823852539, 11.809062957763672, 11.751981735229492, 9.836803436279297, 11.149360656738281, 8.552565574645996, 19.53312110900879, 9.40467357635498, 57.837440490722656, 127.60533905029297, 162.45936584472656, 10.049362182617188, 13.753044128417969, 9.91958999633789, 10.679210662841797, 14.733678817749023, 16.162046432495117, 14.738016128540039, 8.688590049743652, 9.488587379455566, 10.390073776245117, 12.220230102539062, 17.97164535522461, 11.917936325073242, 12.77354621887207, 122.51663970947266, 13.018320083618164, 29.500389099121094, 13.695216178894043, 7.677901268005371, 11.925130844116211, 11.63265609741211, 14.598740577697754, 8.793184280395508, 9.61007308959961, 15.646742820739746, 10.85749626159668, 9.581891059875488, 6.957205772399902, 9.169694900512695, 8.084449768066406, 13.764472961425781, 11.417728424072266, 13.520153999328613, 8.476621627807617, 14.583221435546875, 16.36006736755371, 9.060769081115723, 14.622052192687988, 8.669107437133789, 15.453374862670898, 66.88666534423828, 14.979894638061523, 39.97492980957031, 9.534529685974121, 7.574397087097168, 105.84112548828125, 103.65167999267578, 11.514229774475098, 15.024741172790527, 11.263084411621094, 10.627811431884766, 110.02336120605469, 13.266741752624512, 11.310781478881836, 14.431439399719238, 11.647754669189453, 12.899818420410156, 12.66507339477539, 7.01042366027832, 103.96430206298828, 9.226714134216309, 10.979680061340332, 10.29652214050293, 15.284788131713867, 10.112447738647461, 8.08627986907959, 9.527101516723633, 13.342375755310059, 10.244726181030273, 53.834556579589844, 18.168251037597656, 10.023539543151855, 9.41650390625, 8.872298240661621, 11.116047859191895, 13.809286117553711, 14.377636909484863, 11.273211479187012, 15.296149253845215, 19.633121490478516, 15.473773956298828, 15.747109413146973, 8.09081745147705, 14.680644035339355, 155.7126007080078, 18.38865852355957, 48.13482666015625, 17.592416763305664, 17.83753776550293, 13.947685241699219, 13.696029663085938, 13.184452056884766, 9.349638938903809, 17.270761489868164, 17.364730834960938, 16.005722045898438, 11.558378219604492, 15.170513153076172, 8.704754829406738, 7.988996982574463, 14.980242729187012, 15.65312671661377, 12.198525428771973, 12.585460662841797, 16.34124755859375, 10.13729476928711, 9.479806900024414, 74.33616638183594, 12.941319465637207, 50.59339904785156, 11.591085433959961, 18.349679946899414, 85.08485412597656, 63.94694900512695, 11.460451126098633, 15.610684394836426, 15.32370662689209, 13.70503044128418, 12.134138107299805, 13.400908470153809, 34.701290130615234, 8.55223274230957, 8.251972198486328, 10.261795043945312, 15.639007568359375, 12.463747024536133, 8.603158950805664, 13.986992835998535, 12.255403518676758, 136.52333068847656, 69.43120574951172, 18.251461029052734, 132.99876403808594, 138.3623046875, 9.267549514770508, 12.359795570373535, 15.092620849609375, 15.888463973999023, 12.654233932495117, 11.973995208740234, 10.652603149414062, 17.84035873413086, 63.87723159790039, 8.830561637878418, 6.5758891105651855, 15.27479076385498, 164.5474395751953, 13.285260200500488, 12.185795783996582, 35.95561599731445, 9.471367835998535, 17.47522735595703, 9.32912540435791, 16.654827117919922, 12.83366870880127, 11.214512825012207, 10.654961585998535, 12.735363960266113, 20.18677520751953, 19.344646453857422, 17.80190086364746, 8.988260269165039, 12.921379089355469, 66.90129852294922, 12.185553550720215, 19.175600051879883, 77.3653793334961, 15.233799934387207, 9.528573036193848, 9.220916748046875, 8.536723136901855, 10.196846961975098, 7.074995517730713, 11.425490379333496, 37.67750930786133, 12.21656322479248, 11.064606666564941, 29.703685760498047, 11.45055103302002, 12.22926139831543, 15.208965301513672, 13.68692398071289, 12.310434341430664, 13.840001106262207, 12.784908294677734, 12.615533828735352, 21.69209098815918, 153.73052978515625, 11.688406944274902, 12.223881721496582, 8.931360244750977, 54.17787551879883, 26.591575622558594, 150.6623992919922, 12.215184211730957, 11.22672176361084, 14.542119979858398, 11.332563400268555, 43.920650482177734, 14.328064918518066, 12.229255676269531, 18.254159927368164, 11.667840003967285, 13.106279373168945, 61.104915618896484, 54.35823059082031, 9.9403076171875, 47.03205490112305, 16.357715606689453, 16.120464324951172, 19.938329696655273, 12.886302947998047, 10.002044677734375, 95.0712890625, 118.82674407958984, 23.3905029296875, 14.119189262390137, 9.82382869720459, 10.526959419250488, 14.379056930541992, 9.73119068145752, 13.662481307983398, 11.844560623168945, 12.650840759277344, 18.912405014038086, 12.958770751953125, 17.221914291381836, 16.449764251708984, 14.057931900024414, 11.456040382385254, 9.597614288330078, 11.03891372680664, 15.047831535339355, 14.62671184539795, 9.400816917419434, 9.998135566711426, 9.248537063598633, 15.15690803527832, 8.428330421447754, 12.958674430847168, 11.353193283081055, 9.019954681396484, 5.470017433166504, 89.22689819335938, 14.959040641784668, 11.163580894470215, 15.594826698303223, 10.155869483947754, 15.759013175964355, 56.14876174926758, 8.824878692626953, 7.858039379119873, 14.362342834472656, 30.219619750976562, 12.62021541595459, 11.552582740783691, 10.594474792480469, 10.085614204406738, 74.32196044921875, 12.948049545288086, 11.009873390197754, 12.462905883789062, 9.460677146911621, 12.462874412536621, 26.163347244262695, 9.916387557983398, 14.37231159210205, 203.8240509033203, 12.37559700012207, 15.859240531921387, 11.507948875427246, 10.275510787963867, 12.488435745239258, 8.174215316772461, 11.58873462677002, 12.419869422912598, 59.667381286621094, 16.184667587280273, 16.4028377532959, 170.38026428222656, 18.946910858154297, 8.54996109008789, 14.630195617675781, 10.946823120117188, 16.125104904174805, 13.093369483947754, 100.4333267211914, 7.538882255554199, 12.547657012939453, 46.0038948059082, 18.024150848388672, 13.398458480834961, 9.880167007446289, 23.833572387695312, 12.325243949890137, 11.065940856933594, 13.795522689819336, 14.252900123596191, 17.775192260742188, 12.410799980163574, 16.469419479370117, 11.72106647491455, 18.22467803955078, 8.736921310424805, 111.95013427734375, 175.4748992919922, 16.06268882751465, 9.399003028869629, 17.490232467651367, 7.957008361816406, 12.764958381652832, 133.76708984375, 15.781551361083984, 13.744386672973633, 9.796845436096191, 142.2766876220703, 16.08262825012207, 166.6785888671875, 16.741594314575195, 12.11180305480957, 16.3627986907959, 12.054116249084473, 194.31260681152344, 14.79419231414795, 15.087723731994629, 14.175556182861328, 22.302406311035156, 12.934561729431152, 24.184831619262695, 17.32135009765625, 18.252546310424805, 21.929767608642578, 20.605636596679688, 14.797348022460938, 9.09424877166748, 20.58714485168457, 8.17387580871582, 16.363861083984375, 72.96472930908203, 15.878890991210938, 8.010947227478027, 13.428961753845215, 8.052207946777344, 11.052301406860352, 15.765710830688477, 13.031343460083008, 15.82046127319336, 14.471246719360352, 11.223167419433594, 124.21110534667969, 12.346597671508789, 12.312237739562988, 17.0380859375, 48.81864547729492, 29.63359832763672, 14.074021339416504, 16.644628524780273, 12.293562889099121, 11.583462715148926, 16.19434356689453, 16.715150833129883, 11.53654670715332, 13.840137481689453, 15.457473754882812, 13.123186111450195, 17.069332122802734, 11.399835586547852, 142.8853759765625, 17.379364013671875, 11.871139526367188, 9.094378471374512, 9.039986610412598, 13.43899917602539, 49.03520202636719, 6.575032711029053, 133.71739196777344, 15.60612964630127, 13.089861869812012, 16.087383270263672, 12.263043403625488, 15.989480018615723, 162.14952087402344, 14.667198181152344, 55.6801872253418, 8.931838989257812, 13.639259338378906, 17.064550399780273, 8.437634468078613, 8.656922340393066, 17.9226131439209, 14.941658020019531, 18.089466094970703, 14.67471694946289, 15.266197204589844, 13.511316299438477, 15.27334213256836, 8.715910911560059, 8.36967658996582, 14.459037780761719, 10.390020370483398, 12.0615816116333, 13.569972038269043, 84.26110076904297, 14.742437362670898, 18.557884216308594, 17.26433753967285, 16.859169006347656, 13.981230735778809, 13.444422721862793, 178.5700225830078, 16.49481964111328, 11.771444320678711, 10.947118759155273, 19.490629196166992, 9.113893508911133, 17.141969680786133, 9.937722206115723, 10.135127067565918, 10.040139198303223, 7.973670482635498, 10.53572940826416, 9.812070846557617, 7.384104251861572, 11.37843132019043, 13.294004440307617, 12.72232723236084, 52.50181198120117, 12.091665267944336, 13.786208152770996, 17.591262817382812, 12.970584869384766, 16.18998146057129, 14.79277515411377, 14.558652877807617, 15.50737190246582, 11.221904754638672, 9.377652168273926, 11.53473949432373, 11.190836906433105, 10.285490036010742, 12.048312187194824, 8.760688781738281, 12.907904624938965, 220.3480682373047, 111.8046875, 9.865073204040527, 11.575908660888672, 15.246135711669922, 7.72969388961792, 15.844488143920898, 11.990882873535156, 38.82966613769531, 11.085688591003418, 12.844738006591797, 9.811701774597168, 17.542123794555664, 14.383271217346191, 64.61996459960938, 9.461498260498047, 16.489360809326172, 7.376523017883301, 65.7966537475586, 15.599047660827637, 13.16312313079834, 15.470519065856934, 47.49458312988281, 16.18788719177246, 15.53671646118164, 9.30184268951416, 16.957809448242188, 7.7273268699646, 9.142633438110352, 12.374739646911621, 7.5265212059021, 157.52890014648438, 12.837890625, 14.004963874816895, 15.630797386169434, 13.929664611816406, 11.323504447937012, 50.720848083496094, 11.876067161560059, 13.670981407165527, 13.156845092773438, 10.852157592773438, 15.625970840454102, 10.76753044128418, 12.650374412536621, 14.132311820983887, 17.63429832458496, 8.651337623596191, 17.2716007232666, 11.232564926147461, 12.533136367797852, 11.003175735473633, 14.636994361877441, 11.33708381652832, 91.03730773925781, 40.02381134033203, 9.097562789916992, 16.979145050048828, 12.676946640014648, 9.422231674194336, 11.337764739990234, 13.014565467834473, 16.965206146240234, 12.387036323547363, 15.396544456481934, 6.564858913421631, 71.87628936767578, 8.013175010681152, 8.242096900939941, 12.435206413269043, 147.5908966064453, 138.42652893066406, 114.30470275878906, 10.839818000793457, 16.290802001953125, 13.592391967773438, 14.190210342407227, 16.246055603027344, 142.59092712402344, 11.441048622131348, 54.891998291015625, 14.500266075134277, 21.419105529785156, 17.798555374145508, 15.622515678405762, 13.263020515441895, 9.332393646240234, 10.802836418151855, 85.75786590576172, 15.58056926727295, 18.124895095825195, 14.351917266845703, 8.005956649780273, 9.081339836120605, 9.951499938964844, 13.07180404663086, 9.945802688598633, 10.003525733947754, 19.182592391967773, 15.112751007080078, 10.81812858581543, 16.670732498168945, 10.106057167053223, 17.51632308959961, 15.381536483764648, 11.057509422302246, 14.146021842956543, 16.244054794311523, 11.45511531829834, 13.781841278076172, 10.74805736541748, 10.913025856018066, 27.925058364868164, 19.464279174804688, 11.539408683776855, 9.6166410446167, 14.0327730178833, 17.005638122558594, 10.695758819580078, 10.960856437683105, 12.931998252868652, 14.182735443115234, 16.13882827758789, 10.542652130126953, 13.57353687286377, 11.279426574707031, 116.06900024414062, 10.4847412109375, 18.625417709350586, 15.394287109375, 12.967584609985352, 12.814960479736328, 16.246234893798828, 16.60116195678711, 11.960012435913086, 11.07629680633545, 10.25995922088623, 13.684603691101074, 18.604572296142578, 13.894072532653809, 19.31411361694336, 8.928311347961426, 11.464178085327148, 17.91940689086914, 5.673206329345703, 48.86235809326172, 15.145599365234375, 14.614830017089844, 14.354720115661621, 8.045815467834473, 12.76467514038086, 15.633538246154785, 115.26212310791016, 50.57384490966797, 90.33570861816406, 14.281891822814941, 13.993602752685547, 15.816133499145508, 16.091108322143555, 8.680500030517578, 10.122669219970703, 13.896780967712402, 11.28520393371582, 122.37181091308594, 6.807218551635742, 9.024900436401367, 12.887194633483887, 9.139013290405273, 80.26177215576172, 8.12682819366455, 13.743145942687988, 18.034664154052734, 18.528079986572266, 28.51078224182129, 11.51089859008789, 11.374303817749023, 13.881119728088379, 19.976898193359375, 14.466131210327148, 15.204961776733398, 187.2611541748047, 9.675200462341309, 12.856583595275879, 11.766061782836914, 84.6542739868164, 14.962515830993652, 8.959504127502441, 90.68121337890625, 14.667311668395996, 14.964252471923828, 13.448004722595215, 11.970993995666504, 13.908254623413086, 20.0065860748291, 12.905213356018066, 13.188199996948242, 16.407289505004883, 13.770527839660645, 10.660563468933105, 11.364012718200684, 14.72529125213623, 14.776281356811523, 9.951301574707031, 10.699148178100586, 16.44202423095703, 12.054954528808594, 14.398064613342285, 17.108966827392578, 165.95721435546875, 20.91140365600586, 17.055418014526367, 15.977522850036621, 11.35595703125, 11.429590225219727, 14.858479499816895, 12.002546310424805, 18.466215133666992, 10.067083358764648, 11.017016410827637, 13.94614315032959, 13.276747703552246, 10.109062194824219, 12.62822437286377, 9.777125358581543, 11.79808235168457, 17.664905548095703, 9.126642227172852, 13.857895851135254, 10.955305099487305, 18.9017276763916, 10.195615768432617, 10.90639877319336, 10.529989242553711, 14.129281044006348, 14.657818794250488, 15.377714157104492, 60.84510040283203, 45.424293518066406, 40.75861740112305, 15.45384407043457, 11.610026359558105, 14.479411125183105, 12.139348030090332, 13.499317169189453, 102.27689361572266, 13.122955322265625, 7.852864742279053, 12.398476600646973, 14.181758880615234, 14.26103687286377, 15.71291732788086, 14.042689323425293, 10.397858619689941, 12.511665344238281, 11.73927116394043, 14.34234619140625, 14.526444435119629, 9.600144386291504, 11.002616882324219, 19.589147567749023, 14.650925636291504, 10.451865196228027, 14.249784469604492, 8.13686466217041, 15.104304313659668, 12.04511547088623, 16.024261474609375, 16.164026260375977, 134.5271453857422, 13.368378639221191, 12.499334335327148, 5.926703453063965, 12.326188087463379, 16.284189224243164, 9.589702606201172, 25.124393463134766, 14.183710098266602, 11.710257530212402, 139.70675659179688, 9.075762748718262, 18.563488006591797, 10.069012641906738, 14.833630561828613, 14.340484619140625, 34.56134796142578, 11.69694995880127, 17.472375869750977, 13.537954330444336, 14.534211158752441, 19.59473991394043, 12.861031532287598, 11.104040145874023, 16.074113845825195, 13.680458068847656, 18.2030086517334, 12.956439018249512, 12.566363334655762, 119.60617065429688, 15.785053253173828, 12.658767700195312, 12.87319278717041, 58.949127197265625, 16.63397789001465, 82.85777282714844, 83.54594421386719, 7.514512538909912, 14.835630416870117, 14.058816909790039, 14.46037483215332, 12.559333801269531, 11.035896301269531, 14.26414680480957, 9.32264232635498, 11.189698219299316, 16.805620193481445, 16.648969650268555, 9.514270782470703, 7.781946659088135, 10.127595901489258, 10.958752632141113, 14.904104232788086, 9.770278930664062, 14.329120635986328, 10.087492942810059, 9.317648887634277, 12.296387672424316]


In [ ]:

# # Convert the list to a numpy array
# distances_array = np.array(distances_random_graphs)

# # Calculate the z-scores for each element in the array
# z_scores = (distances_array - np.mean(distances_array)) / np.std(distances_array)

# # Define a threshold for outliers (e.g., z-score > 3)
# threshold = 0.5

# # Create a mask to identify outliers
# outlier_mask = np.abs(z_scores) > threshold

# # Remove outliers from the array
# filtered_distances = distances_array[~outlier_mask]

# # Convert the filtered array back to a list
# filtered_distances_list = filtered_distances.tolist()

In [ ]:
# sum(filtered_distances_list) / len(filtered_distances_list)

13.445761892345402

In [ ]:
# filtered_distances_list

[18.90079116821289,
 13.994956016540527,
 18.59467315673828,
 99.57367706298828,
 12.549628257751465,
 12.131684303283691,
 33.354164123535156,
 12.624557495117188,
 7.622239589691162,
 7.140101909637451,
 8.129176139831543,
 11.115296363830566,
 11.150999069213867,
 12.553301811218262,
 18.150859832763672,
 39.439090728759766,
 14.725622177124023,
 12.986281394958496,
 9.173707962036133,
 11.731589317321777,
 10.623479843139648,
 14.190513610839844,
 9.719880104064941,
 22.595802307128906,
 11.12348747253418,
 16.891321182250977,
 15.867040634155273,
 6.8220624923706055,
 15.32767391204834,
 14.361724853515625,
 13.085883140563965,
 9.035792350769043,
 11.632275581359863,
 20.091903686523438,
 15.532211303710938,
 117.51826477050781,
 12.189861297607422,
 16.283920288085938,
 18.390846252441406,
 8.840758323669434,
 15.95419692993164,
 9.660080909729004,
 8.623753547668457,
 17.921588897705078,
 12.339995384216309,
 10.192910194396973,
 14.382380485534668,
 17.58751678466797,
 8.23885

In [ ]:
cosine_similarities_random_graphs = [-0.5733916759490967, -0.18243595957756042, -0.7121228575706482, 0.5937678813934326, 0.012442097067832947, 0.26442885398864746, -0.3610403835773468, 0.04477282613515854, 0.6790544986724854, 0.7297958731651306, 0.6751742362976074, 0.3321571946144104, 0.1388016641139984, 0.05877023562788963, -0.48067331314086914, -0.6611607670783997, -0.09902279824018478, -0.07521085441112518, 0.5548652410507202, 0.14968666434288025, 0.392704576253891, -0.25216543674468994, 0.45017239451408386, 0.7222474813461304, 0.09121093153953552, 0.17910760641098022, -0.7242927551269531, -0.5300548076629639, 0.7556641101837158, 0.05112136900424957, -0.07199403643608093, 0.05111892521381378, 0.5012452006340027, -0.015263665467500687, 0.23035767674446106, -0.28973859548568726, -0.6694125533103943, -0.07532153278589249, -0.3977583050727844, -0.5259963274002075, 0.5555919408798218, -0.4443911910057068, 0.44408705830574036, 0.5592018365859985, -0.5831881761550903, 0.30270153284072876, 0.3252179026603699, -0.4503931701183319, -0.6730148196220398, 0.6236386895179749, -0.24566729366779327, 0.4111860990524292, 0.13535824418067932, 0.10201156884431839, -0.604465901851654, -0.5591543912887573, 0.26942744851112366, 0.45014864206314087, 0.3160128593444824, 0.5342301726341248, -0.009671628475189209, 0.1561911702156067, 0.5795995593070984, -0.09718988835811615, -0.7776961326599121, -0.15703044831752777, 0.5164393186569214, 0.6226742267608643, -0.6494823694229126, 0.3104826807975769, -0.6747108101844788, 0.6730862855911255, 0.4779793620109558, 0.5914679169654846, -0.1621818095445633, 0.5859533548355103, 0.6537377834320068, -0.5058891177177429, 0.30705681443214417, -0.6049225926399231, -0.5050442814826965, 0.05452120304107666, 0.4827822148799896, -0.11553338170051575, -0.2971646189689636, 0.5876870155334473, 0.5373468399047852, -0.38681143522262573, 0.1856442391872406, -0.274341344833374, 0.8042550683021545, 0.7158863544464111, -0.7814508676528931, 0.5961471796035767, 0.14391380548477173, -0.4823943078517914, 0.7390779852867126, 0.22076129913330078, -0.21578344702720642, -0.04807520657777786, -0.3769935369491577, 0.12254104018211365, -0.6667158603668213, -0.4262106418609619, 0.03277086466550827, -0.012140560895204544, -0.7849697470664978, -0.635300874710083, 0.1743384748697281, 0.2692382335662842, -0.4548723101615906, 0.0998510867357254, -0.7952581644058228, 0.5674362182617188, 0.5500462651252747, -0.5236456394195557, -0.0809079185128212, 0.1276634931564331, 0.158916175365448, 0.6301069259643555, 0.4790336489677429, 0.19481953978538513, -0.5100100040435791, -0.39633116126060486, 0.025682218372821808, -0.22802290320396423, -0.2288340926170349, 0.5562891364097595, -0.4420315623283386, 0.43951135873794556, -0.24332529306411743, 0.5483733415603638, -0.6787289381027222, 0.15359936654567719, -0.07105885446071625, 0.5860909819602966, 0.11534848809242249, -0.6613373756408691, 0.35165154933929443, -0.14020636677742004, -0.6845802068710327, -0.0908070057630539, -0.3610624074935913, 0.32230955362319946, 0.576387882232666, 0.06547150015830994, 0.3848092257976532, -0.41572561860084534, -0.33320316672325134, -0.3061363697052002, -0.08066026866436005, -0.012170910835266113, 0.6738044619560242, -0.3614068627357483, -0.18746110796928406, 0.6377118229866028, 0.34194403886795044, 0.19724714756011963, -0.6048839092254639, -0.44798341393470764, -0.5744189620018005, 0.031265124678611755, 0.2026987373828888, 0.31198760867118835, -0.23697441816329956, 0.06218980997800827, 0.02764519676566124, 0.4121990501880646, 0.07768827676773071, 0.5408139228820801, 0.5991456508636475, -0.03470119833946228, 0.05110868066549301, 0.24781298637390137, 0.31784671545028687, 0.6076951026916504, 0.2679022550582886, 0.12670598924160004, -0.12439819425344467, 0.195135697722435, -0.03422228991985321, 0.12421279400587082, 0.2805449962615967, 0.0114840567111969, 0.21619845926761627, -0.004610203206539154, 0.5911303758621216, 0.44781994819641113, -0.351080060005188, 0.5412566661834717, -0.5927373170852661, -0.21409538388252258, 0.09423041343688965, -0.39630159735679626, -0.5732524394989014, 0.011081010103225708, 0.22716446220874786, 0.1618889570236206, 0.25655195116996765, 0.09739190340042114, -0.3877996504306793, -0.20535436272621155, -0.09417194128036499, 0.4736090898513794, 0.5991295576095581, 0.5719828605651855, 0.05720297992229462, 0.7369128465652466, -0.34861844778060913, -0.2043343186378479, -0.30436742305755615, 0.12173036485910416, 0.10265523940324783, 0.790286660194397, -0.6270272731781006, -0.7218108177185059, 0.3480603098869324, -0.36575859785079956, -0.5219361782073975, -0.6569912433624268, 0.11715736985206604, 0.7073639631271362, -0.16704398393630981, 0.4519181251525879, 0.18924346566200256, -0.043157435953617096, 0.5038586258888245, -0.02649320662021637, 0.013839498162269592, -0.029577337205410004, -0.0638756975531578, 0.4354686439037323, 0.059895191341638565, -0.03325249254703522, -0.3628406524658203, 0.30443891882896423, -0.14527860283851624, 0.6531492471694946, 0.09719515591859818, -0.3943752646446228, 0.8242390155792236, -0.4329882562160492, 0.5580894351005554, 0.8103373050689697, -0.012911483645439148, 0.2075975239276886, 0.1377120018005371, -0.4803919196128845, -0.13831296563148499, -0.30422234535217285, 0.27279266715049744, 0.17275701463222504, 0.09371287375688553, -0.4389985203742981, -0.08884197473526001, 0.47270894050598145, -0.5515432953834534, -0.009087227284908295, -0.531948447227478, 0.5232331156730652, -0.038164276629686356, -0.795231819152832, -0.291805624961853, 0.04909740388393402, 0.5933061838150024, -0.6211322546005249, -0.07641825079917908, 0.285785973072052, -0.39875853061676025, -0.6305197477340698, 0.7742120623588562, 0.38897955417633057, -0.4829724431037903, -0.650458812713623, 0.41016560792922974, 0.012169450521469116, 0.05922199785709381, 0.4104481637477875, -0.2599504888057709, -0.1978803128004074, -0.31989941000938416, 0.3892238736152649, 0.2659674286842346, 0.10578754544258118, -0.5101363658905029, 0.21080954372882843, -0.5675592422485352, 0.2779005467891693, 0.2547367811203003, -0.11902520805597305, 0.844881534576416, -0.7575527429580688, 0.14218643307685852, 0.38869789242744446, -0.6136040687561035, 0.3462160527706146, 0.616876482963562, -0.8047798871994019, -0.01595873013138771, -0.1661473512649536, 0.17185498774051666, 0.5110495686531067, 0.48636916279792786, -0.5170131325721741, 0.7163554430007935, -0.5069226026535034, -0.6099916696548462, 0.6170599460601807, 0.3399513065814972, -0.26295092701911926, 0.3097904920578003, 0.002405606210231781, 0.7310841083526611, 0.4432848393917084, -0.16644906997680664, -0.22093385457992554, 0.07549157738685608, -0.03793218731880188, 0.2994093894958496, -0.11038213968276978, 0.691786527633667, -0.5276148319244385, 0.10078790038824081, 0.037228621542453766, 0.08749054372310638, -0.183679461479187, 0.08325307071208954, -0.3872694969177246, -0.16971950232982635, 0.25231754779815674, 0.22860217094421387, 0.10588456690311432, -0.1995369791984558, 0.7520353198051453, 0.6197415590286255, 0.6336115598678589, 0.22905153036117554, -0.15472018718719482, -0.17405816912651062, 0.5359117984771729, 0.6023842096328735, 0.5551953315734863, -0.07687705010175705, -0.37259912490844727, 0.28753507137298584, -0.43469178676605225, -0.2632153630256653, -0.2626485824584961, -0.14056961238384247, -0.3203786611557007, -0.3048520088195801, 0.4280923008918762, 0.22386467456817627, -0.5340561866760254, 0.31107550859451294, 0.05024312064051628, 0.5712265968322754, 0.16587980091571808, 0.6265099048614502, -0.5734971761703491, 0.7544304132461548, -0.12136473506689072, 0.11647752672433853, 0.5508955717086792, 0.6107521653175354, -0.1451471894979477, 0.368429958820343, 0.18305493891239166, -0.6679341197013855, 0.6554140448570251, -0.28561538457870483, -0.35893192887306213, 0.38161635398864746, 0.018479937687516212, 0.4347611665725708, -0.1350063979625702, 0.6747368574142456, 0.2856869101524353, 0.17155101895332336, 0.08535384386777878, -0.5279488563537598, -0.04883567616343498, -0.16086649894714355, -0.18447329103946686, 0.11980970203876495, -0.12768056988716125, 0.22866466641426086, 0.6565190553665161, -0.19490650296211243, 0.5255292654037476, 0.42244964838027954, -0.6801179051399231, 0.5632802248001099, 0.27292826771736145, 0.3618333041667938, -0.23460453748703003, 0.6693099737167358, 0.6299563646316528, -0.5213680267333984, -0.3785417973995209, 0.42249536514282227, -0.3267105221748352, -0.13577522337436676, -0.006887644529342651, -0.2811451852321625, 0.3018132448196411, -0.2707175016403198, 0.020268738269805908, -0.13116784393787384, 0.35169312357902527, 0.38611334562301636, -0.5234402418136597, 0.3456171154975891, -0.3416057229042053, -0.04620338976383209, -0.5905121564865112, 0.08922375738620758, 0.2642447352409363, 0.17608550190925598, -0.7845210433006287, 0.18584808707237244, -0.712889552116394, 0.29760175943374634, -0.20636966824531555, 0.07863286882638931, -0.16836607456207275, 0.7165167927742004, 0.14545176923274994, 0.4753078818321228, -0.09417473524808884, 0.6642269492149353, 0.23262977600097656, -0.11772002279758453, 0.30276304483413696, -0.3596224784851074, 0.5429558753967285, 0.8685895204544067, 0.6577094793319702, 0.15576115250587463, -0.13265544176101685, -0.0026279985904693604, -0.27360737323760986, 0.3574044108390808, -0.2801033854484558, 0.6228601336479187, -0.6828722357749939, 0.7905670404434204, 0.5412846207618713, -0.39201635122299194, -0.5517081022262573, -0.27986735105514526, -0.42274174094200134, 0.21071743965148926, 0.4019443988800049, -0.26117104291915894, 0.6045259237289429, -0.5839898586273193, -0.1757066696882248, -0.006222188472747803, 0.16491015255451202, 0.02626320719718933, -0.6453303694725037, 0.33775052428245544, 0.45344382524490356, 0.4616885781288147, -0.6436724662780762, -0.5609480738639832, 0.17303195595741272, 0.4099103808403015, 0.6119431853294373, 0.6591236591339111, 0.5883287787437439, -0.6745706796646118, -0.6673614978790283, 0.27711743116378784, -0.5163062214851379, -0.1404837667942047, -0.18866074085235596, -0.20370806753635406, -0.42164739966392517, 0.07413151115179062, -0.008428052067756653, 0.7747581601142883, -0.04832059144973755, -0.06297548860311508, 0.6301627159118652, 0.09113997220993042, 0.2854207754135132, 0.08014945685863495, -0.7054903507232666, -0.14359746873378754, 0.7330209016799927, 0.1463392823934555, -0.651931643486023, 0.23102302849292755, -0.34274739027023315, 0.23794902861118317, -0.25651806592941284, -0.09657220542430878, -0.70259690284729, 0.3402349650859833, 0.2555345296859741, -0.40220221877098083, -0.5637511014938354, 0.2965984046459198, 0.5324985384941101, 0.41397765278816223, -0.4935489594936371, -0.502967357635498, 0.09837954491376877, 0.18343903124332428, 0.2583574056625366, -0.020892299711704254, -0.11263291537761688, 0.5788379311561584, -0.35389718413352966, 0.4557361602783203, 0.38983649015426636, 0.023945782333612442, -0.26582831144332886, -0.2080971598625183, -0.727271318435669, -0.4090704619884491, -0.6181954145431519, 0.3273448348045349, 0.05330432951450348, -0.4259741008281708, 0.6598981618881226, 0.26357609033584595, 0.6216409206390381, 0.3790319263935089, -0.20643413066864014, 0.5656565427780151, -0.571442186832428, 0.22765620052814484, 0.26275211572647095, -0.5980613827705383, -0.3294038772583008, 0.6880114674568176, 0.017368905246257782, 0.18037447333335876, -0.24808913469314575, 0.15125827491283417, -0.5469595789909363, -0.225815087556839, 0.5220126509666443, -0.12015236169099808, -0.057508647441864014, -0.330228716135025, 0.09240385890007019, -0.5122676491737366, 0.22126896679401398, 0.007066348567605019, -0.2300392985343933, -0.5497246980667114, 0.7863456010818481, 0.5445704460144043, 0.06751873344182968, 0.5690628886222839, -0.8619457483291626, -0.2922631502151489, 0.16087979078292847, -0.6253011226654053, -0.43838733434677124, 0.5014797449111938, -0.14199841022491455, 0.2616601884365082, 0.19702407717704773, -0.44846558570861816, -0.048120055347681046, -0.6749567985534668, 0.09984326362609863, -0.6115096807479858, 0.06495146453380585, 0.21631640195846558, 0.49963122606277466, 0.3354448676109314, 0.12524671852588654, -0.5384572744369507, -0.03520745784044266, -0.22042426466941833, 0.50980544090271, -0.24361273646354675, 0.1033700555562973, 0.2936234772205353, 0.7627062797546387, 0.5285990834236145, 0.26958608627319336, 0.17728370428085327, -0.15594226121902466, -0.3910635709762573, -0.005524665117263794, 0.7130957841873169, 0.06753101944923401, 0.15277314186096191, -0.6209707856178284, -0.0837676003575325, -0.43317851424217224, -0.6594445705413818, 0.6560956835746765, 0.08972009271383286, 0.2697829008102417, 0.6227679252624512, -0.5141441822052002, 0.3131110668182373, 0.04372979700565338, 0.4182049632072449, 0.605547308921814, 0.4558274745941162, -0.030708029866218567, -0.019477706402540207, 0.3309882879257202, 0.635122537612915, -0.41372397541999817, -0.4971112906932831, -0.4666368365287781, -0.2277635633945465, 0.5340476036071777, -0.2023196965456009, -0.6161906123161316, 0.39730316400527954, 0.02022428810596466, -0.44988390803337097, -0.26573044061660767, 0.15165925025939941, -0.15491348505020142, -0.5241444706916809, -0.3238222301006317, 0.10130812227725983, 0.1607888638973236, -0.09431244432926178, 0.1702250987291336, -0.5883992910385132, 0.8598611354827881, 0.4160524606704712, -0.8423501253128052, 0.44041386246681213, -0.2870364189147949, 0.2906234562397003, 0.7332508563995361, -0.5802329778671265, -0.5957018136978149, 0.36511945724487305, -0.2575112283229828, -0.16402576863765717, 0.5892684459686279, 0.17715927958488464, 0.1264856457710266, 0.19282139837741852, 0.14343494176864624, -0.18227283656597137, 0.6818991303443909, 0.39711302518844604, -0.05442528426647186, 0.7461692094802856, -0.6036603450775146, -0.2618674039840698, 0.4039912819862366, -0.10793647170066833, 0.7056435346603394, 0.2367997020483017, 0.2302357256412506, 0.26587700843811035, 0.502135157585144, 0.06398150324821472, -0.3131771683692932, 0.08412755280733109, 0.06148197874426842, -0.28228193521499634, 0.3022286891937256, 0.6410159468650818, 0.5912692546844482, -0.8099882006645203, 0.5853114724159241, -0.024095263332128525, 0.1555081307888031, 0.44973617792129517, -0.5963664054870605, 0.2847662568092346, 0.47801703214645386, 0.18659117817878723, 0.30630427598953247, -0.029921483248472214, 0.38448646664619446, 0.115599624812603, -0.6564061641693115, -0.25089725852012634, 0.6536480188369751, 0.48958665132522583, 0.08669228851795197, 0.6099149584770203, 0.4393446743488312, 0.47532767057418823, -0.47226375341415405, 0.06001898646354675, -0.23957911133766174, -0.07316409051418304, -0.08523533493280411, 0.5680112838745117, -0.5147450566291809, 0.4251687526702881, 0.6811525821685791, 0.656383752822876, -0.0018234960734844208, -0.11787445843219757, 0.22028231620788574, 0.13459354639053345, -0.4170660674571991, 0.5982263088226318, 0.4971954822540283, -0.5398616790771484, 0.5256651639938354, 0.06673724949359894, 0.5247305035591125, -0.16074225306510925, 0.04256046190857887, -0.21073243021965027, -0.2664584219455719, 0.3831610381603241, -0.12847481667995453, -0.5351990461349487, -0.21002954244613647, 0.2735428810119629, 0.4205118715763092, 0.6659680604934692, -0.1467364877462387, 0.057203516364097595, 0.6354217529296875, -0.5192978978157043, -0.055667147040367126, 0.06452947854995728, 0.17746946215629578, -0.0015057474374771118, 0.020904581993818283, 0.6022471189498901, -0.6070519685745239, 0.06351067125797272, 0.5548730492591858, 0.13400417566299438, 0.5637155175209045, -0.3619762659072876, 0.12776470184326172, 0.03920316696166992, -0.5690755844116211, 0.051008448004722595, -0.3110150098800659, -0.35747963190078735, -0.45562314987182617, 0.4085933566093445, 0.7870115637779236, 0.7118613123893738, 0.3307780623435974, 0.44368624687194824, 0.5147162675857544, 0.20709142088890076, -0.28203731775283813, -0.5185396671295166, -0.45758914947509766, -0.12890155613422394, 0.5079526901245117, 0.29072800278663635, 0.09928493946790695, 0.35485172271728516, 0.6147537231445312, 0.11582601070404053, -0.2734176814556122, 0.514870285987854, -0.34441515803337097, -0.2752876877784729, 0.4080295264720917, -0.50787353515625, 0.002758108079433441, 0.449130654335022, -0.14845845103263855, 0.005068781785666943, -0.5502643585205078, 0.1815641075372696, -0.47859615087509155, 0.4677145779132843, -0.387024462223053, 0.765792965888977, -0.4831824004650116, -0.18839068710803986, -0.36298978328704834, 0.37616750597953796, -0.6716566681861877, 0.48078879714012146, -0.14754974842071533, 0.5756008625030518, 0.5684263110160828, -0.02219535782933235, -0.47364991903305054, 0.6184468269348145, 0.5837214589118958, -0.37644240260124207, -0.16048555076122284, -0.5487934350967407, -0.6667330265045166, 0.44291990995407104, -0.5366073250770569, -0.22909805178642273, 0.16739454865455627, -0.29703667759895325, 0.22398635745048523, 0.002982914447784424, -0.0164735559374094, -0.7126708030700684, -0.23172666132450104, -0.10128867626190186, 0.07010817527770996, 0.6710719466209412, -0.3783101737499237, 0.478383868932724, -0.28527209162712097, -0.11484493315219879, 0.45434924960136414, 0.22719953954219818, -0.21143899857997894, 0.16624072194099426, 0.5381746888160706, -0.6194303035736084, 0.4900560975074768, 0.6743534207344055, -0.4688907563686371, -0.5984084010124207, 0.519681453704834, 0.33039700984954834, -0.12357237190008163, 0.5511194467544556, -0.4881719946861267, 0.26482900977134705, 0.8343052268028259, 0.010702528059482574, 0.06828880310058594, -0.15253853797912598, -0.7157600522041321, 0.3825128376483917, 0.36354899406433105, 0.48805513978004456, 0.8260020017623901, 0.02576986700296402, -0.25665247440338135, 0.20961163938045502, -0.7795047760009766, -0.32791775465011597, -0.5422888398170471, -0.5624762177467346, 0.6994055509567261, 0.3871007263660431, 0.09729477763175964, 0.34803956747055054, -0.4013925790786743, -0.17711177468299866, -0.6020183563232422, 0.5549136996269226, 0.529502272605896, -0.3092550039291382, 0.5939677953720093, -0.47761252522468567, 0.6110949516296387, -0.5865945816040039, 0.3868263065814972, -0.396164208650589, -0.04813970625400543, -0.030690491199493408, 0.046386778354644775, -0.5222496390342712, -0.12768256664276123, -0.22792625427246094, -0.37668347358703613, 0.09820342063903809, -0.19651401042938232, 0.025678366422653198, 0.5104801654815674, 0.10690382122993469, -0.12925615906715393, -0.20073716342449188, 0.2700728178024292, 0.4249573349952698, -0.15943264961242676, 0.09469317644834518, 0.7498675584793091, 0.41550204157829285, 0.4973025321960449, 0.01397009938955307, 0.032303452491760254, 0.466726690530777, 0.46819809079170227, -0.09530456364154816, 0.27027854323387146, 0.7202824354171753, -0.40229159593582153, 0.43648701906204224, -0.20437635481357574, 0.1789703667163849, 0.21799686551094055, 0.26995015144348145, 0.028801079839468002, 0.5873542428016663, 0.25523027777671814, 0.5600002408027649, 0.3359502851963043, -0.48838406801223755, 0.30828312039375305, 0.44487255811691284, 0.11064980924129486, 0.3947003483772278, 0.40010973811149597, -0.47795775532722473, 0.12425004690885544, 0.20066192746162415, -0.4520930051803589, -0.3283848166465759, 0.7208623886108398, 0.27112579345703125, 0.7370313405990601, 0.12794163823127747, 0.7711842656135559, 0.2374330312013626, 0.04179364815354347, 0.11395209282636642, 0.10072237253189087, -0.4098420739173889, 0.24171999096870422, 0.10433360189199448, -0.40240478515625, 0.31027668714523315, -0.2124185860157013, 0.4400312304496765, -0.5526827573776245, -0.5790392756462097, 0.46421387791633606, 0.2138761281967163, -0.32649505138397217, 0.2045571506023407, -0.6175879240036011, -0.4814785122871399, -0.5603128671646118, 0.17654810845851898, -0.5011122226715088, 0.4783777892589569, 0.5031583905220032, 0.685967206954956, 0.17073741555213928, -0.5798283219337463, 0.0745270848274231, -0.03544144332408905, 0.6173440217971802, -0.5235661268234253, -0.07326890528202057, 0.6491314768791199, -0.4556739330291748, -0.1131221354007721, 0.7278019785881042, -0.13864682614803314, -0.15610414743423462, 0.13383185863494873, 0.09201346337795258, 0.11426977068185806, 0.19991210103034973, -0.02441626787185669, 0.17054663598537445, 0.5656821131706238, -0.8419822454452515, 0.46149253845214844, -0.28917908668518066, 0.07880860567092896, -0.07382065802812576, -0.17736797034740448, -0.4943279027938843, 0.49303382635116577, -0.3679599463939667, -0.5366345047950745, 0.37368011474609375, 0.5054812431335449, -0.1199369728565216, 0.4904419779777527, -0.18840652704238892, 0.773412823677063, 0.46351003646850586, -0.046744972467422485, 0.360984206199646, -0.35960543155670166, 0.3762366473674774, 0.023405462503433228, -0.4139786958694458, 0.013356611132621765, -0.1452821046113968, 0.6055119037628174, 0.36998143792152405, 0.2981550395488739, -0.6888143420219421, -0.2221640944480896, 0.46275103092193604, -0.14347553253173828, 0.8095307350158691, 0.3550710380077362, 0.6310954093933105, 0.7508705854415894, -0.26180481910705566, 0.549472451210022, 0.4013616442680359, 0.17339330911636353, -0.3887444734573364, 0.41237473487854004, 0.4645969867706299, 0.13233277201652527, -0.18161441385746002, -0.5415073037147522, 0.2455410659313202, 0.11217524856328964, 0.0902208611369133, 0.4159790277481079, -0.08593092858791351, 0.5440750122070312, 0.24992720782756805, -0.09558019042015076, -0.03481335937976837, 0.18475155532360077, -0.5111662149429321, -0.37667450308799744, -0.10209439694881439, -0.29278820753097534, -0.04188679903745651, 0.5813899636268616, 0.4155276417732239, -0.2049618810415268, 0.260662317276001, 0.6210891008377075, -0.17456763982772827, 0.6063819527626038, -0.011218288913369179, -0.0960179790854454, 0.27693086862564087, 0.33464768528938293, -0.464019238948822, 0.2983498275279999, 0.3086208701133728, -0.35417070984840393, -0.20775945484638214, 0.41846656799316406, -0.371328204870224, 0.027458809316158295, -0.1138855367898941, -0.7396658658981323, -0.09678767621517181, 0.6775652766227722, -0.6208010911941528, 0.471809983253479, -0.41409364342689514, 0.08480771631002426, -0.41999003291130066, 0.19076578319072723, -0.13977736234664917, 0.6173905730247498, -0.07857759296894073, -0.15361179411411285, -0.4686545133590698, 0.4125106930732727, 0.49045467376708984, -0.1605762392282486, 0.4318535327911377, -0.13186578452587128, 0.5925627946853638, -0.6395581364631653, -0.5573068261146545, 0.5678240656852722, 0.14599257707595825, 0.3357333242893219, 0.47097247838974, -0.34445518255233765, 0.36051061749458313, 0.08994419872760773, 0.06583768129348755, 0.6709885597229004, 0.16957589983940125, 0.5482898950576782, -0.12308073788881302, 0.23392333090305328, -0.6763837337493896, -0.7033684253692627, -0.4143126308917999, -0.2940109670162201, -0.1540324091911316, 0.736217737197876, -0.5613383650779724, 0.5482355952262878, 0.08489539474248886, -0.31537264585494995, 0.12835414707660675, -0.21560697257518768, 0.4645771384239197, 0.15595154464244843, 0.40803778171539307, -0.7098769545555115, -0.200198233127594, 0.16310305893421173, 0.26596367359161377, 0.3335285186767578, 0.474838525056839, 0.5774622559547424, 0.45062047243118286, -0.010357733815908432, -0.7543002367019653, 0.6997140645980835, 0.4769672453403473, 0.3745526075363159, -0.23345127701759338, -0.7419165372848511, 0.2902021110057831, 0.2514769732952118, 0.2508704960346222, 0.37095189094543457, 0.47854378819465637, 0.6464582681655884, 0.09074147045612335, 0.24531583487987518, 0.037869811058044434, -0.24201133847236633, 0.2443842738866806, 0.17008058726787567, 0.1260484755039215, -0.6428366303443909, -0.6201655864715576, 0.4577328860759735, -0.28926852345466614, 0.4256014823913574, -0.399069607257843, 0.4063451290130615, -0.5811585187911987, 0.3221220374107361, 0.19962018728256226, 0.6269357204437256, -0.1339876353740692, -0.12401449680328369, 0.07332421839237213, -0.018688775599002838, 0.02240375429391861, -0.16630609333515167, -0.2887554466724396, -0.38943931460380554, 0.3202686905860901, 0.630408763885498, -0.42751458287239075, 0.2605595290660858, 0.11830972135066986, 0.5781424045562744, -0.5152105093002319, 0.7294799089431763, -0.28132474422454834, 0.395155131816864, -0.13404451310634613, -0.7035855054855347, 0.3905099928379059, -0.4183432161808014, -0.2888336777687073, 0.28011444211006165, -0.6617365479469299, -0.508886456489563, 0.7412952184677124, -0.35625895857810974, 0.6361326575279236, -0.046129826456308365, 0.08791220188140869, 0.05934895575046539, 0.5421205163002014, -0.5150974988937378, 0.1149974912405014, -0.557364821434021, -0.29207056760787964, 0.4726350009441376, -0.4037007987499237, -0.5269558429718018, 0.07668045908212662, 0.687827467918396, 0.2695983648300171, 0.2307063639163971, 0.5932895541191101, 0.7128434181213379, -0.4015064239501953, 0.4108864665031433, 0.4674128293991089, -0.07251212745904922, 0.6312634944915771, -0.16171936690807343, 0.09799230098724365, 0.3279629051685333, -0.2196725606918335, 0.21131914854049683, -0.8044320344924927, 0.2979353666305542, -0.614625871181488, -0.42211711406707764, -0.24416233599185944, 0.3618486523628235, -0.5051220655441284, 0.3187938928604126, 0.5367901921272278, 0.4066007435321808, -0.7477941513061523, 0.7542401552200317, -0.34951362013816833, 0.49559786915779114, -0.25053244829177856, 0.6245971322059631, -0.07517530769109726, -0.14710983633995056, 0.6773128509521484, -0.1392192840576172, -0.19568464159965515, -0.11067691445350647, -0.6668269634246826, -0.03281155228614807, -0.19307860732078552, -0.5305639505386353, -0.09016595035791397, 0.5531688332557678, 0.41712671518325806, -0.7729340195655823, 0.7503901720046997, -0.702305793762207, 0.21770504117012024, 0.33153945207595825, 0.4191617965698242, 0.397554874420166, 0.570902407169342, 0.17517556250095367, 0.4810984432697296, -0.3055311441421509, -0.7350434064865112, 0.2172461450099945, 0.3792915940284729, -0.05037274956703186, 0.4726070165634155, 0.40636134147644043, -0.1857604831457138, -0.5798765420913696, -0.40350210666656494, 0.5597034692764282, 0.5055335760116577, 0.45920562744140625, 0.18195420503616333, -0.38872212171554565, 0.024517107754945755, -0.28601428866386414, -0.006743639707565308, 0.012753628194332123, 0.2843526303768158, -0.28559136390686035, 0.6995722651481628, 0.1288398653268814, 0.27685803174972534, -0.08657754957675934, 0.566604733467102, 0.4740287959575653, -0.3217947483062744, 0.20502802729606628, 0.5354629755020142, 0.7502778172492981, 0.4834918677806854, 0.6292842626571655, -0.04231932759284973, 0.17959025502204895, -0.06373439729213715, 0.6783856749534607, -0.21998834609985352, -0.3898496627807617, 0.4988575577735901, 0.04932743310928345, 0.5506458282470703, -0.2897045910358429, -0.7039303779602051, -0.42558836936950684, 0.11745622754096985, 0.4438628554344177, 0.6841111183166504, 0.6454013586044312, 0.4470122158527374, 0.24377086758613586, -0.044303253293037415, 0.23790687322616577, 0.3980141878128052, -0.039017852395772934, -0.09985415637493134, 0.23554982244968414, -0.22691699862480164, 0.008145347237586975, 0.06540609896183014, 0.10243716835975647, 0.7387874126434326, 0.23793253302574158, 0.5441722869873047, 0.25517383217811584, 0.4104451835155487, -0.21539324522018433, 0.531017541885376, 0.6434940099716187, 0.4184821844100952, -0.0756806880235672, 0.3573858141899109, 0.830612301826477, -0.04173550009727478, 0.4953729808330536, 0.5534889698028564, 0.5405205488204956, 0.21051403880119324, -0.2663462162017822, -0.37370753288269043, 0.36412307620048523, -0.3338313400745392, -0.6909642219543457, -0.41456592082977295, 0.2312082052230835, 0.6267166137695312, 0.11043401807546616, 0.5037618279457092, -0.6114948987960815, -0.07297732681035995, -0.6044229865074158, -0.6302546262741089, -0.3517509698867798, -0.32134777307510376, 0.07135482877492905, 0.5784188508987427, -0.6777678728103638, -0.3693031072616577, -0.47522038221359253, 0.16883622109889984, -0.5701640844345093, 0.5501623153686523, 0.6703283190727234, 0.02994273602962494, -0.3708263039588928, 0.006084367632865906, -0.1465369462966919, -0.3787311911582947, 0.3307918906211853, 0.46733319759368896, 0.051109325140714645, -0.10923223197460175, -0.7568206191062927, -0.035668082535266876, -0.47440803050994873, -0.7960994243621826, -0.7974162697792053, 0.23305389285087585, -0.23586030304431915, -0.17277374863624573, -0.1146586686372757, -0.09992866963148117, -0.1867983043193817, 0.5079618692398071, 0.5740235447883606, 0.6316779851913452, 0.28443649411201477, -0.22160376608371735, 0.19076687097549438, 0.5682718753814697, -0.3794693648815155, 0.32333287596702576, 0.6389250755310059, 0.6246658563613892, -0.6861845254898071, -0.04013041406869888, 0.8531829118728638, 0.49892544746398926, -0.07101541012525558, -0.4546143412590027, -0.11056546866893768, 0.023851178586483, 0.3409140706062317, 0.2048511952161789, -0.28444409370422363, -0.41783249378204346, 0.5524609088897705, 0.7742438912391663, -0.07611510157585144, -0.6217145919799805, 0.05162392556667328, -0.06511207669973373, -0.2776217460632324, 0.4348011016845703, -0.37844717502593994, 0.5342006087303162, -0.43652990460395813, 0.08994437009096146, 0.2793450355529785, 0.16924279928207397, -0.07315637171268463, -0.6954290866851807, -0.4445217251777649, -0.4878159761428833, 0.5625643730163574, 0.12312446534633636, 0.6204774379730225, 0.1921558976173401, -0.24872419238090515, 0.7305166721343994, 0.13974350690841675, 0.406904399394989, 0.4876870810985565, 0.5743679404258728, 0.3343506157398224, 0.7322220206260681, 0.16007405519485474, -0.09433934837579727, -0.06223684921860695, 0.24411222338676453, -0.7409727573394775, 0.24507838487625122, 0.14075252413749695, -0.10792134702205658, -0.05274321511387825, -0.011042959988117218, -0.2622985243797302, -0.23272447288036346, 0.14439179003238678, -0.5265233516693115, 0.7149235606193542, 0.1745346486568451, 0.018544919788837433, 0.5126016139984131, -0.2128673493862152, 0.3040766417980194, -0.6122360229492188, -0.04321156442165375, 0.19791628420352936, -0.1895926296710968, 0.16546553373336792, 0.7561722993850708, -0.11231310665607452, -0.1447073370218277, -0.4420621395111084, -0.006592210382223129, -0.3100642263889313, -0.6272932291030884, 0.21358852088451385, 0.3389129042625427, 0.7753542065620422, -0.4661829471588135, -0.47113049030303955, -0.7168704271316528, -0.06204424798488617, 0.5046127438545227, 0.38278669118881226, 0.13514089584350586, -0.27336356043815613, -0.14423984289169312, 0.3782803416252136, 0.44102463126182556, -0.32370251417160034, 0.4917706251144409, -0.1334211826324463, 0.32536518573760986, -0.09936198592185974, -0.7436571717262268, 0.11949638277292252, -0.6573895812034607, -0.16424885392189026, -0.39867103099823, 0.1727592945098877, 0.4827808141708374, 0.15921586751937866, -0.13690681755542755, -0.38714057207107544, 0.5126180648803711, 0.3993379473686218, 0.4717872738838196, -0.41173917055130005, 0.6359071731567383, -0.21421296894550323, 0.09428692609071732, 0.6068346500396729, 0.852388322353363, 0.15192914009094238, -0.6018966436386108, 0.3235432803630829, -0.28276705741882324, 0.3321838080883026, -0.15183481574058533, -0.14455726742744446, 0.5544477701187134, 0.6718169450759888, -0.21825315058231354, -0.07021360099315643, 0.026558227837085724, 0.08432185649871826, 0.3073168396949768, 0.37912529706954956, -0.027018077671527863, -0.35325583815574646, 0.31330394744873047, 0.048726361244916916, 0.43235671520233154, 0.20252105593681335, -0.39660701155662537, 0.4598173499107361, -0.4692857265472412, -0.2868615984916687, -0.052779801189899445, -0.2996641993522644, 0.20191335678100586, 0.335460901260376, 0.4449325203895569, 0.6169196367263794, 0.34249147772789, 0.38455039262771606, -0.06302812695503235, -0.24174872040748596, -0.5299979448318481, 0.4945681095123291, -0.4738926887512207, 0.5815935730934143, 0.36662280559539795, 0.3262489438056946, -0.24130739271640778, -0.24343068897724152, 0.43792009353637695, 0.7015775442123413, -0.21066653728485107, -0.27976319193840027, -0.6011287569999695, -0.29228442907333374, 0.4199526309967041, -0.18218472599983215, 0.07205038517713547, 0.3342123031616211, -0.33356404304504395, -0.2152702808380127, -0.5376983284950256, -0.0925799012184143, -0.41851264238357544, 0.2642202079296112, -0.45529550313949585, 0.6228127479553223, 0.4977905750274658, 0.319117933511734, -0.496845543384552, 0.4444032907485962, -0.5082234144210815, 0.6526924967765808, 0.11499937623739243, 0.15591134130954742, 0.01221703365445137, 0.020118458196520805, 0.3744571805000305, 0.28497612476348877, -0.3652070462703705, -0.0441603884100914, -0.2630593478679657, 0.13222995400428772, -0.28055739402770996, 0.23486605286598206, 0.4640158712863922, -0.3981165885925293, -0.27142107486724854, -0.25994250178337097, -0.7740183472633362, -0.04884082078933716, -0.5978678464889526, -0.6120924949645996, -0.6184090375900269, 0.14960822463035583, 0.07556381821632385, -0.22524338960647583, 0.5521577000617981, -0.5069746375083923, 0.6178854703903198, -0.5826621055603027, -0.7751685976982117, -0.5555651187896729, 0.6362115144729614, -0.11115739494562149, 0.6423659324645996, 0.39893126487731934, -0.4069422483444214, 0.3623955249786377, -0.4352869987487793, -0.2851431965827942, 0.2578031122684479, 0.651231050491333, -0.10586702078580856, 0.11605864763259888, -0.5919453501701355, -0.055032677948474884, 0.7587589025497437, -0.024173401296138763, -0.3449074625968933, -0.08907327055931091, 0.17588958144187927, -0.1973648965358734, -0.0376959852874279, 0.20976726710796356, -0.3164973258972168, -0.5664467811584473, -0.3309916853904724, -0.5963999032974243, 0.08586487174034119, -0.6087281107902527, -0.4265872836112976, 0.029620632529258728, 0.5999453663825989, 0.5219807028770447, -0.1044454574584961, 0.7636626958847046, 0.7757887840270996, -0.4238440692424774, 0.257015585899353, 0.20228266716003418, -0.42292529344558716, 0.23352137207984924, -0.2974838316440582, -0.19151374697685242, -0.1839752495288849, -0.048918500542640686, 0.5331356525421143, 0.17149196565151215, -0.5601609349250793, 0.5900737643241882, 0.5532001256942749, -0.5918225049972534, -0.1933830976486206, -0.556270182132721, -0.21548524498939514, -0.3084610104560852, 0.03055228665471077, -0.5578950643539429, 0.5805261135101318, 0.6013265252113342, -0.5490957498550415, 0.7406992316246033, 0.1946500837802887, -0.06551198661327362, -0.383123517036438, -0.3366684317588806, -0.30596357583999634, -0.515376091003418, -0.5491725206375122, -0.32514089345932007, -0.1279819905757904, 0.3831940293312073, -0.4804195761680603, 0.227277934551239, 0.23410747945308685, -0.5241787433624268, 0.4950331449508667, -0.4045273959636688, 0.3510221540927887, 0.38670623302459717, 0.3751099705696106, 0.6407225131988525, 0.2800210118293762, 0.5399073362350464, 0.7045600414276123, 0.014826266095042229, -0.06005753576755524, 0.17164918780326843, -0.34696120023727417, 0.07395988702774048, -0.061988674104213715, -0.5554199814796448, 0.1251739263534546, -0.593913197517395, -0.33316242694854736, -0.1876063495874405, -0.08210378885269165, 0.07141992449760437, 0.45317792892456055, 0.17231601476669312, 0.2222408801317215, 0.3370918929576874, -0.036390140652656555, 0.7388077974319458, 0.082400843501091, -0.7130030393600464, -0.07665836066007614, 0.37570691108703613, 0.03436138853430748, -0.20261140167713165, 0.6837809085845947, -0.46457281708717346, -0.10465659946203232, -0.23609326779842377, 0.24218402802944183, 0.3901659846305847, 0.418690025806427, -0.505260169506073, -0.35908544063568115, 0.20631064474582672, 0.47035402059555054, -0.36611223220825195, 0.7038753032684326, -0.3539855480194092, 0.29433971643447876, 0.07298032194375992, -0.5401085615158081, -0.46646398305892944, -0.41917896270751953, -0.5537427663803101, 0.4843412935733795, -0.5907701253890991, 0.6683430075645447, 0.5168402194976807, 0.03665061295032501, 0.6999272108078003, 0.28982922434806824, 0.12835605442523956, -0.12769950926303864, -0.600927472114563, 0.29631882905960083, 0.2165224701166153, -0.6605216264724731, 0.1586374044418335, -0.2494320571422577, -0.2720521092414856, 0.46894535422325134, -0.3171488642692566, 0.3624594807624817, -0.06829105317592621, -0.13525474071502686, -0.38857072591781616, 0.6104930639266968, -0.24728889763355255, 0.30061641335487366, -0.03721725940704346, 0.35409775376319885, -0.37608852982521057, 0.0777035802602768, 0.3500838577747345, 0.29658156633377075, 0.7294199466705322, -0.6192507743835449, 0.1470959484577179, 0.4515478014945984, 0.36482423543930054, 0.10355249047279358, -0.5001461505889893, -0.018627099692821503, -0.3768444061279297, 0.7747223973274231, 0.631510317325592, 0.6387085914611816, 0.6210192441940308, -0.14964967966079712, -0.6431471705436707, 0.29039961099624634, -0.7501845359802246, 0.3191096782684326, -0.5087363719940186, -0.20246709883213043, -0.12723274528980255, -0.6597743034362793, 0.23602475225925446, 0.14942055940628052, 0.8391011357307434, -0.3159404993057251, -0.18332375586032867, -0.5463655591011047, -0.4716203212738037, -0.07321162521839142, 0.4785920977592468, 0.2519766390323639, 0.2775716185569763, -0.5830998420715332, -0.4081937074661255, -0.21335935592651367, 0.705878734588623, 0.5511264801025391, 0.400968462228775, 0.06400547921657562, 0.4072932004928589, 0.4537653923034668, -0.272466242313385, -0.035646021366119385, 0.46821165084838867, -0.40629738569259644, 0.36646556854248047, -0.47416090965270996, 0.04473671317100525, 0.3684307336807251, -0.07967016100883484, -0.48730260133743286, 0.06949688494205475, -0.22634975612163544, 0.20517507195472717, 0.1481073498725891, 0.19428685307502747, -0.4823806881904602, 0.2602171003818512, 0.41786813735961914, -0.2230600118637085, -0.6583885550498962, 0.3756077289581299, 0.265247642993927, -0.026696426793932915, -0.3902190029621124, -0.30638808012008667, 0.32276344299316406, -0.19996806979179382, 0.1824459284543991, 0.5603129267692566, 0.39028340578079224, -0.457768976688385, -0.3618382215499878, -0.13194015622138977, -0.10514570772647858, -0.5483117699623108, -0.36712372303009033, 0.22030529379844666, 0.09812962263822556, 0.40359970927238464, -0.021911174058914185, -0.48557889461517334, -0.3965533375740051, -0.5517196655273438, 0.5196229815483093, 0.23061218857765198, -0.5268370509147644, 0.8492172956466675, -0.25321558117866516, -0.23051825165748596, -0.14937034249305725, -0.2690052092075348, 0.652308464050293, -0.1865862011909485, -0.5136537551879883, -0.7260504961013794, -0.4978369474411011, -0.7641294002532959, -0.22205260396003723, 0.10882444679737091, -0.36365896463394165, -0.6958153247833252, 0.5953055620193481, 0.4101647436618805, -0.14761601388454437, 0.4094293415546417, 0.7144374847412109, 0.755272388458252, 0.5291697382926941, -0.11792628467082977, 0.528895378112793, -0.7644875049591064, 0.6358593702316284, -0.30551767349243164, -0.6288837790489197, -0.45531684160232544, -0.3743770718574524, 0.2360847145318985, 0.21840310096740723, -0.30159875750541687, 0.21850040555000305, -0.2602810263633728, -0.6212258338928223, 0.6638138294219971, 0.47177982330322266, -0.2364300787448883, 0.34719881415367126, -0.4713566303253174, -0.3894243836402893, 0.5132153034210205, -0.7071176767349243, -0.37985992431640625, -0.25178617238998413, -0.3342225253582001, 0.031148258596658707, -0.1334829181432724, -0.16223600506782532, 0.07278066128492355, -0.2864242196083069, -0.512043297290802, -0.26085418462753296, 0.2098533809185028, 0.10511104762554169, -0.2790307104587555, 0.017964929342269897, 0.389814168214798, 0.44503867626190186, -0.450128972530365, 0.2170540988445282, -0.15369008481502533, -0.3113363981246948, -0.8307949304580688, -0.0801195502281189, -0.6656486988067627, -0.599091649055481, 0.07154914736747742, 0.19995418190956116, -0.5422928333282471, 0.4125281274318695, -0.6445891857147217, 0.3303329348564148, 0.2878987789154053, -0.07639506459236145, -0.12218062579631805, 0.30932748317718506, -0.09411265701055527, 0.608311116695404, 0.17806299030780792, -0.6274670362472534, 0.4779926836490631, -0.22998343408107758, 0.18822216987609863, -0.6829530596733093, 0.3883100152015686, 0.15929603576660156, 0.23843914270401, -0.3562888503074646, -0.262732595205307, -0.37563034892082214, 0.5145342350006104, 0.785926342010498, 0.1357758641242981, -0.3743242621421814, 0.30324238538742065, -0.22320546209812164, -0.008660629391670227, 0.1366375982761383, -0.750240683555603, -0.19638502597808838, 0.7037400007247925, -0.09603821486234665, -0.08042605221271515, -0.24391473829746246, -0.5284572839736938, -0.3556906580924988, 0.22562526166439056, -0.27184224128723145, 0.28233665227890015, -0.16583657264709473, -0.4920257329940796, 0.4318465292453766, 0.2706912159919739, -0.6279051303863525, -0.2554212808609009, 0.41594114899635315, -0.3343530595302582, 0.6323733329772949, -0.18191862106323242, 0.07206549495458603, -0.4747363328933716, -0.6122041940689087, 0.4725205898284912, -0.32534754276275635, 0.4427485764026642, 0.8222391605377197, -0.11955840140581131, 0.11692583560943604, 0.48213547468185425, -0.07972502708435059, -0.064659982919693, 0.020458459854125977, -0.6733010411262512, 0.5209726095199585, -0.4239923059940338, 0.3190804719924927, -0.19955219328403473, -0.09544743597507477, 0.5177631378173828, 0.13063207268714905, -0.6469107270240784, -0.001178830862045288, -0.06421887874603271, -0.06611607223749161, -0.18001142144203186, 0.4067516028881073, -0.44815072417259216, -0.09120939671993256, 0.02379429340362549, -0.21375831961631775, 0.14830641448497772, 0.12350906431674957, -0.3305020332336426, -0.15916840732097626, -0.051727451384067535, 0.3601769208908081, -0.709111213684082, -0.3037310540676117, 0.8250676393508911, 0.6918914318084717, -0.29923325777053833, -0.01152036339044571, -0.26484012603759766, 0.1358112096786499, 0.21626117825508118, -0.2749255299568176, 0.4735207259654999, 0.2571769952774048, -0.5410444736480713, -0.32707467675209045, 0.4573764503002167, 0.6679362058639526, 0.4637390375137329, 0.2706890404224396, 0.014664016664028168, 0.44287896156311035, -0.4767298698425293, 0.44046905636787415, 0.46421578526496887, 0.15999408066272736]


In [ ]:
# # Convert the list to a numpy array
# cos_sim_array = np.array(cosine_similarities_random_graphs)

# # Calculate the z-scores for each element in the array
# z_scores = (cos_sim_array - np.mean(cos_sim_array)) / np.std(cos_sim_array)

# # Define a threshold for outliers (e.g., z-score > 3)
# threshold = 0.5

# # Create a mask to identify outliers
# outlier_mask = np.abs(z_scores) > threshold

# # Remove outliers from the array
# filtered_cos_sim = cos_sim_array[~outlier_mask]

# # Convert the filtered array back to a list
# filtered_cos_sim_list = filtered_cos_sim.tolist()

In [ ]:
# sum(filtered_cos_sim_list) / len(filtered_cos_sim_list)

0.03925594690674916